<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FINAL_GNN_FINE_TUNINGT_T2SQL_DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries Installation

In [1]:
!pip install datasets networkx -q

!pip install torch_geometric -q


# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 , L4  IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet


!pip install peft --quiet
!pip install trl ninja packaging --quiet
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

!pip install mistral_inference -q

!pip install trl==0.8.6 -q


!pip install sqlparse -q

!pip install bitsandbytes -q

#!pip uninstall -y torchvision -q
!pip install torchvision --no-cache-dir -q
#import evaluate

!pip install sentence-transformers -q

!pip install nlpaug -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset

from peft import LoraConfig, get_peft_model, TaskType

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)

#from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import spacy
import numpy as np

from torch_geometric.nn import GAT

from trl import setup_chat_format

import colab_env
import evaluate

Mounted at /content/gdrive


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1"


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model and tokenizer
mistral_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## TRAININING

https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Mistral_7B_Instruct_v0_1_for_text_to_SQL_EVALDATA.ipynb

* Import Main Components

In [ ]:
!pip cache purge

Files removed: 3076


In [ ]:
train_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dataset[0]['messages'][0]['content']

'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_92 (total VARCHAR, finish VARCHAR)'

In [ ]:
train_dataset[0]['messages'][1]['content']

'How many times was the finish t32?'

In [ ]:
train_dataset[0]['messages'][2]['content']

'SELECT COUNT(total) FROM table_name_92 WHERE finish = "t32"'

In [ ]:
len(train_dataset)

10000

* https://stackoverflow.com/questions/70950706/assertionerror-in-torch-geometric-nn-gatconv


1. **Data Input and Preparation:**
   - The process begins by loading the "sql-create-context" dataset, which presumably contains pairs of natural language questions and their corresponding SQL queries.
   - The dataset is divided into three distinct subsets: training, validation, and testing.
   - The Mistral-7B-Instruct-v0.3 language model and its tokenizer are loaded and prepared.

2. **Data Transformation with `TextToSQLDataset`:**
   - The `TextToSQLDataset` class is responsible for converting raw data into a format suitable for model training and evaluation.
   - For each data sample, the following transformations occur:
      - Tokenization: The input question and the SQL query (answer) are tokenized using the loaded tokenizer.
      - Dependency Parsing: The question is parsed using spaCy to extract the grammatical relationships between words, generating a dependency graph.
      - Dictionary Creation: A dictionary is created to store the tokenized input IDs, attention masks, labels (tokenized SQL query), and the dependency edges extracted from parsing.

3. **Batching and Shuffling with `DataLoader`:**
   - `DataLoader` takes the processed dataset from `TextToSQLDataset` and creates an iterable object for efficient batching.
   - Optionally, shuffling is applied to randomize the order of samples within each epoch during training.

4. **Forward Pass through `GraphModel`:**
   - **Mistral Encoder:** The tokenized input IDs and attention masks are fed into the Mistral model's encoder to obtain contextualized token embeddings.
   - **GATv2 Layer:** The GATv2 layer (Graph Attention Network) takes the token embeddings and the dependency edges as input. It applies graph attention mechanisms to incorporate the structural information from the dependency graph into the token representations.
   - **Pooling:** The node representations (output of GATv2) are aggregated using a pooling operation (e.g., mean pooling) to obtain a fixed-size representation of the entire input sequence.
   - **LM Head:** The pooled representation is passed through a linear layer, which produces logits – unnormalized probabilities for each token in the vocabulary.
   - **Loss Calculation:** During training, if labels (correct SQL queries) are available, the cross-entropy loss is calculated between the predicted logits and the true labels. This loss guides the optimization of the model's parameters.

5. **Model Optimization with PEFT (LoRA):**
   - The `GraphModel` is wrapped with PEFT's LoRA (Low-Rank Adaptation) configuration to enable parameter-efficient fine-tuning.
   - During training, only the parameters of the GATv2 layer and the LM head are updated, while the rest of the model parameters remain frozen.
   - The Hugging Face Trainer manages the training process, iterating over the dataset, computing gradients, and updating the model's parameters based on the calculated loss.

6. **Evaluation:**
   - After (or during) training, the model is evaluated on the validation and test sets.
   - The `compute_metrics` function decodes the predicted logits and labels back into text and assesses the model's performance using two metrics:
      - Semantic Similarity: This metric measures how semantically close the predicted SQL query is to the reference SQL query using SentenceTransformer embeddings.
      - Exact Match: This metric checks if the predicted SQL query matches the reference SQL query exactly.

7. **Output:**
    - The final output is the evaluation results, including semantic similarity and exact match scores, which provide insights into the model's ability to generate accurate SQL queries from natural language questions.


The corrected graphical representation of the dataflow, with the arrow pointing *into* the `Trainer`:

```
+-------------------+          +----------------------+
| Dataset Loading   |          | TextToSQLDataset     |
|  - sql-create...  | -------> | - Tokenization       |
|  - Split: train...|          | - Dependency Parsing |
+-------------------+          | - Dict Creation      |
                               +----------------------+
                                         |
                                         v
                         +---------------------------+
                         | DataLoader                |
                         | - Batches, Shuffling (opt)|
                         +---------------------------+
                                         |
                                         v
                         +---------------------------+
                         | GraphModel                |
                         | - Mistral Encoder         |
                         | - GATv2 Layer             |
                         | - Pooling                 |
                         | - LM Head                 |
                         | - (Loss Calculation)      |
                         +---------------------------+
                                         |
                                         v
            +--------------+          +---------+         +-----------------+
            | PEFT (LoRA)  | -------> | Trainer | <------ |Evaluation       |
            +--------------+          |         |         |(compute_metrics)|
                                      +---------+         +-----------------+
                                         |
                                         v
                                   +-----------------------+
                                   | - Semantic Similarity |
                                   | - Exact Match         |
                                   +-----------------------+
```

The revised dataflow now accurately shows that the evaluation metrics (semantic similarity and exact match) calculated by the `compute_metrics` function are used by the `Trainer` to assess the model's performance and make decisions during training (e.g., early stopping).



In [ ]:
import huggingface_hub
import shutil

# Get the cache directory information
cache_info = huggingface_hub.utils.scan_cache_dir()

# Iterate through the repositories and delete them
for repo_info in cache_info.repos:
    repo_path = repo_info.repo_path
    shutil.rmtree(repo_path)  # Delete the repository folder

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig, Trainer, TrainingArguments,
    set_seed,
    EarlyStoppingCallback,
    DataCollatorForSeq2Seq
)

from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel,
    PeftConfig,
    PrefixTuningConfig,
    PromptEncoderConfig,
    TaskType
)

from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from torch_geometric.nn import GAT
import spacy
import numpy as np
import torch.nn as nn
import evaluate


# Suppress warnings
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore", message="The installed version of bitsandbytes was compiled without GPU support.")

# 1. Load and Prepare Data
dataset = load_dataset("b-mc2/sql-create-context")["train"].shuffle(seed=42)

# Manually define splits
train_size = int(0.7 * len(dataset))
eval_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - eval_size

train_dataset = dataset.select(range(train_size))
eval_dataset = dataset.select(range(train_size, train_size + eval_size))
test_dataset = dataset.select(range(train_size + eval_size, len(dataset)))


# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3a. PyTorch Datasets - Data Augmentation
import random
from nlpaug.augmenter.word import SynonymAug, RandomWordAug

def random_insertion(sentence, aug_p=0.1, synonym_aug=None, max_attempts=3):  # Add max_attempts
    words = sentence.split()
    new_words = []
    for word in words:
        new_words.append(word)
        if random.random() < aug_p:
            attempts = 0
            while attempts < max_attempts:  # Try multiple times to find a synonym
                if new_words:
                    candidate_words = [new_words[-1]]
                if len(new_words) < len(words):
                    candidate_words.append(words[len(new_words)])
                if candidate_words and synonym_aug:
                    synonym = synonym_aug.augment(random.choice(candidate_words))
                    if synonym:
                        new_words.append(synonym[0])
                        break  # Exit the loop if a synonym is found
                attempts += 1
    return ' '.join(new_words)

def augment_data(dataset):
    augmented_data = []

    synonym_aug = SynonymAug(aug_src='wordnet')
    delete_aug = RandomWordAug(action="delete", aug_p=0.05)

    for item in dataset:
        question = item['question']
        answer = item['answer']
        context = item['context']

        augmented_questions = set()
        augmented_questions.add(question)  # Always include the original question

        # Attempt to generate one more unique augmented question
        while len(augmented_questions) < 2:  # Keep trying until we have 2 unique questions
            augmentation_method = random.choice(['synonym', 'insertion', 'deletion'])

            if augmentation_method == 'synonym':
                synonyms = synonym_aug.augment(question)
                if synonyms and synonyms[0] != question:
                    augmented_questions.add(synonyms[0])

            elif augmentation_method == 'insertion':
                augmented_question = random_insertion(question, synonym_aug=synonym_aug)
                if augmented_question != question:
                    augmented_questions.add(augmented_question)

            else:  # deletion
                deleted = delete_aug.augment(question)
                if deleted and deleted[0] != question:
                    augmented_questions.add(deleted[0])

        # Add the augmented examples to the dataset
        for aug_question in list(augmented_questions):
            augmented_data.append({'question': aug_question, 'answer': answer, 'context': context})

    return augmented_data


# 3. PyTorch Datasets
import torch
from torch.utils.data import Dataset
from torch_geometric.data import Data
import spacy

# Load the spaCy English model with SRL capabilities
try:
    nlp = spacy.load("en_core_web_trf")
except OSError:
    spacy.cli.download("en_core_web_trf")
    nlp = spacy.load("en_core_web_trf")

class TextToSQLDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        text = item['question']
        target_text = item['answer']

        # 1. Tokenization
        tokenized_input = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=1024,
            return_tensors="pt"
        )
        tokenized_target = self.tokenizer(
            target_text,
            truncation=True,
            padding="max_length",
            max_length=1024,
            return_tensors="pt"
        )

        # Flatten lists
        tokenized_input = {k: v.squeeze(0) for k, v in tokenized_input.items()}
        tokenized_target = {k: v.squeeze(0) for k, v in tokenized_target.items()}

        # 2. Dependency Parsing & SRL for Edge Extraction
        doc = nlp(text)
        edges = []
        edge_attrs = []
        for token in doc:
            # Dependency Parsing
            if token.dep_ != "ROOT" and token.i != token.head.i:
                edges.append([token.i, token.head.i])
                edge_attrs.append(self.tokenizer.vocab.get(token.dep_, self.tokenizer.unk_token_id))

            # SRL
            if token.dep_ in {"nsubj", "dobj", "nsubjpass"}:
                for child in token.children:
                    if child.dep_ == "prep":
                        for grandchild in child.children:
                            if grandchild.dep_ in {"pobj", "pcomp"}:
                                edges.append([token.i, grandchild.i])
                                edge_attrs.append(self.tokenizer.vocab.get("prep_" + grandchild.dep_, self.tokenizer.unk_token_id))

        # Edge Index Extraction and Validation
        if not edges:
            num_nodes = len(tokenized_input["input_ids"])
            edges = [[i, i] for i in range(num_nodes)]
        else:
            max_index = len(tokenized_input["input_ids"]) - 1
            edges = [(src, tgt) for src, tgt in edges
                     if 0 <= src <= max_index and 0 <= tgt <= max_index]

        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        edge_attrs = torch.tensor(edge_attrs, dtype=torch.long)

        # 3. Node Features with POS Tags
        pos_tags = [token.pos_ for token in doc]
        pos_tag_ids = [self.tokenizer.vocab.get(tag, self.tokenizer.unk_token_id) for tag in pos_tags]
        pos_tag_ids = torch.tensor(pos_tag_ids, dtype=torch.long)

        # Convert everything to tensors BEFORE padding/truncation
        input_ids = tokenized_input["input_ids"].clone().detach()
        attention_mask = tokenized_input["attention_mask"].clone().detach()
        labels = tokenized_target["input_ids"].clone().detach()

        # Handle potentially empty target sequences
        if len(labels) == 0:
            labels = torch.tensor([self.tokenizer.pad_token_id], dtype=torch.long)

        # Padding and Truncation
        max_length = 1024

        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
        pos_tag_ids = pos_tag_ids[:max_length]

        if len(input_ids) < max_length:
            pad_length = max_length - len(input_ids)
            pad_tensor = torch.full((pad_length,), self.tokenizer.pad_token_id)
            input_ids = torch.cat((input_ids, pad_tensor))
            attention_mask = torch.cat((attention_mask, torch.zeros(pad_length, dtype=torch.long)))
            pos_tag_ids = torch.cat((pos_tag_ids, torch.zeros(pad_length, dtype=torch.long)))

        if len(labels) < max_length:
            pad_length = max_length - len(labels)
            labels = torch.cat((labels, torch.full((pad_length,), -100)))

        if len(edge_attrs) < edge_index.size(1):
            pad_length = edge_index.size(1) - len(edge_attrs)
            edge_attrs = torch.cat((edge_attrs, torch.zeros(pad_length, dtype=torch.long)))

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "edges": edge_index,
            "edge_attrs": edge_attrs,
            "pos_tag_ids": pos_tag_ids,
            "sample_ids": torch.tensor([idx])
        }

#Minimum: Start with at least 2000-3000 samples.
#This should be enough to provide a good initial assessment of your model's performance and potential.

#Medium: If your computational resources allow, try 5000-7000 samples.
#This could provide a more robust evaluation and potentially lead to better performance.

#Maximum: If you have ample resources, consider using the entire dataset (around 10,000 samples).
#This would give you the most comprehensive training data possible and potentially lead
#to the best model performance.

#Reduce train_dataset size for POC
#POC_sample=26000

POC_sample=16000
import numpy as np
train_dataset = train_dataset.select(np.random.choice(len(train_dataset), POC_sample, replace=False))

### data augmentation #######
train_dataset_augmented = augment_data(train_dataset)
train_dataset = TextToSQLDataset(train_dataset_augmented, tokenizer)
#############################


POC_valsample=1
#############################
eval_dataset = eval_dataset.select(np.random.choice(len(eval_dataset), POC_valsample, replace=False))
test_dataset = test_dataset.select(np.random.choice(len(test_dataset), POC_valsample, replace=False))

eval_dataset = TextToSQLDataset(eval_dataset, tokenizer)
test_dataset = TextToSQLDataset(test_dataset, tokenizer)
#############################


# 4. GAT Layer and GraphModel
import torch
import torch.nn as nn
from torch_geometric.nn import GAT

class GATLayer(torch.nn.Module):
    def __init__(self, in_features, out_features, num_heads=8, num_layers=3):
        super(GATLayer, self).__init__()
        self.gat = GAT(in_channels=in_features, hidden_channels=out_features, heads=num_heads,
                        concat=False, num_layers=num_layers)

    def forward(self, x, edge_index, edge_attr=None):
        return self.gat(x, edge_index, edge_attr=edge_attr)

    def get_lora_target_modules(self):
        return [module for module in self.gat.modules() if isinstance(module, torch.nn.Linear)]


# 4b.  GraphModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv


from dataclasses import dataclass
from typing import Optional, Tuple

@dataclass
class MyCausalLMOutputWithPast:
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

class GraphModel(nn.Module):
    def __init__(self, encoder, tokenizer):
        super(GraphModel, self).__init__()
        self.encoder = encoder
        self.config = encoder.model.config

        # Adjust in_channels to match the actual input dimensionality
        self.gatv2 = GATv2Conv(
            in_channels=self.config.hidden_size,  # Set to 4096
            out_channels=self.config.hidden_size,
            heads=8,
            concat=False,
        )

        # Max Pooling
        self.pool = lambda x, batch: torch.max(x, dim=0, keepdim=True)[0]

        # Additional Feedforward Layer
        self.ffn = nn.Sequential(
            nn.Linear(self.config.hidden_size, self.config.hidden_size * 2),
            nn.ReLU(),
            nn.Linear(self.config.hidden_size * 2, self.config.hidden_size),
        )

        self.lm_head = nn.Linear(self.config.hidden_size, self.config.vocab_size)
        self.tokenizer = tokenizer

        # Add generation config
        self.generation_config = encoder.generation_config

    def forward(self, input_ids, attention_mask, edges, labels=None, inputs_embeds=None,
                pos_tag_ids=None, edge_attrs=None, sample_ids=None, output_attentions=False,
                output_hidden_states=False, return_dict=False):


        # 1. Token Embeddings (Encoder)
        if input_ids is not None:
            encoder_outputs = self.encoder(
                input_ids.to(self.encoder.model.device),
                attention_mask=attention_mask.to(self.encoder.model.device),
                output_hidden_states=True
            )
            embeddings = encoder_outputs.hidden_states[-1]
        elif inputs_embeds is not None:
            embeddings = inputs_embeds
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        # Ensure correct shape for GATv2Conv
        if embeddings.dim() > 2:
            embeddings = embeddings.view(-1, embeddings.shape[-1])

         # 2. Obtain POS tag embeddings and concatenate
        if pos_tag_ids is not None:
            pos_tag_embeddings = self.encoder.model.embeddings(pos_tag_ids.to(self.encoder.model.device))
            embeddings = torch.cat([embeddings, pos_tag_embeddings], dim=-1)


        # 3. Edge Index Creation (with potential optimization for memory)
        edge_index = []
        node_offset = 0
        for i, graph_edges in enumerate(edges):
            if graph_edges is None or graph_edges.numel() == 0:
                num_nodes = input_ids.size(1)
                graph_edges = torch.arange(node_offset, node_offset + num_nodes, device=embeddings.device)
                graph_edges = graph_edges.repeat(2, 1)
            else:
                if not isinstance(graph_edges, torch.Tensor):
                    graph_edges = torch.tensor(graph_edges, dtype=torch.long, device=embeddings.device)
                graph_edges += node_offset
            edge_index.append(graph_edges)
            node_offset += input_ids.size(1)
        edge_index = torch.cat(edge_index, dim=1)

        # 4. GATv2 Layer (pass edge_attrs)
        graph_out = self.gatv2(embeddings, edge_index, edge_attr=edge_attrs)


        # 5. Pooling
        batch = torch.arange(len(edges), device=graph_out.device).repeat_interleave(input_ids.size(1))
        pooled = self.pool(graph_out, batch).unsqueeze(1)

        # 5.1 - Additional Feedforward Layer
        pooled = self.ffn(pooled)

        # 6. LM Head
        logits = self.lm_head(pooled)

        # 7. Loss Calculation
        loss = None
        if labels is not None:
            from torch.nn import CrossEntropyLoss

            mask = (labels != -100).float()

            # Apply softmax to logits to get probabilities
            log_probs = F.log_softmax(logits, dim=-1)

            loss_fct = CrossEntropyLoss(ignore_index=-100)

            # Reshape log_probs to match target shape
            log_probs = log_probs.squeeze(1)

            labels = labels[:, 0]

            loss = loss_fct(log_probs, labels)

            loss_per_sample = (loss * mask).sum(dim=1) / mask.sum(dim=1)

        # 8. Return
        return {
            "loss": loss,
            "logits": logits,
            "past_key_values": encoder_outputs.past_key_values,
            "hidden_states": encoder_outputs.hidden_states,
            "attentions": encoder_outputs.attentions,
        }

    def prepare_inputs_for_generation(self, input_ids, edges, attention_mask=None,
                                      pos_tag_ids=None, edge_attrs=None, **kwargs):
        if isinstance(self, PeftModel):
            return self.base_model.prepare_inputs_for_generation(input_ids, edges, attention_mask, **kwargs)

        batch_size = input_ids.size(0)
        if batch_size > 1:
            batched_edges = []
            node_offset = 0
            for i in range(batch_size):
                graph_edges = edges[i]
                batched_edges.extend([(src + node_offset, dst + node_offset) for src, dst in graph_edges])
                node_offset += input_ids.size(1)
            edge_index = torch.tensor(batched_edges, dtype=torch.long).t().contiguous().to(input_ids.device)
        else:
            edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous().to(input_ids.device)

        model_inputs = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "edges": edge_index,
            "pos_tag_ids": pos_tag_ids,
            "edge_attrs": edge_attrs,
            "past_key_values": kwargs.get("past_key_values", None),
        }
        return model_inputs

# END GraphModel

from peft import prepare_model_for_kbit_training

# Quantize Mistral (before creating GraphModel)
mistral_model = prepare_model_for_kbit_training(mistral_model, use_gradient_checkpointing=True)


# 5. Model Setup (Define model first)
model = GraphModel(mistral_model, tokenizer)  # Pass both mistral_model and tokenizer


# 6. PEFT Configuration (Use automatic module discovery)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    # Instead of targeting specific layers, let PEFT automatically discover the linear layers within the model
    task_type="CAUSAL_LM",
)


# 7. Apply PEFT
model = get_peft_model(model, peft_config)
print('\n\n')
print('PEFT-Model')
model.print_trainable_parameters() # To see the trainable parameters
print('\n')

# Access the config of the encoder (Mistral model) within your GraphModel
model.encoder.config.use_cache = False

# Ensure that LoRA layers are properly initialized and their dimensions are correctly set
for name, module in model.named_modules():
    if "lora" in name:
        module = module.to(device) # Move LoRA parameters to the correct device

model.encoder.gradient_checkpointing_enable()  # Enable gradient checkpointing for memory optimization on the Mistral model
#model.encoder.model.embed_tokens.requires_grad_(True)
torch.cuda.empty_cache()


# 8. Evaluation Metric (Semantic Similarity)
metric = evaluate.load("exact_match")
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

def compute_metrics(eval_pred):
    all_preds, all_labels = eval_pred

    # Convert all elements to tensors, handling different data types
    predictions = [torch.tensor(pred) if not isinstance(pred, torch.Tensor) else pred for pred in all_preds]
    labels = [torch.tensor(label) if not isinstance(label, torch.Tensor) else label for label in all_labels]

    # Filter out any None values before stacking
    predictions = [pred for pred in predictions if pred is not None]
    labels = [label for label in labels if label is not None]

    # Convert to tensors and stack (only if there are predictions/labels)
    if predictions:
        predictions = torch.stack(predictions).squeeze()
    else:
        predictions = torch.tensor([])  # Empty tensor if no predictions

    if labels:
        labels = torch.stack(labels).squeeze()
    else:
        labels = torch.tensor([])  # Empty tensor if no labels

    # Handle cases where only one prediction/label is present (avoid squeezing to a scalar)
    if predictions.dim() == 0:
        predictions = predictions.unsqueeze(0)
    if labels.dim() == 0:
        labels = labels.unsqueeze(0)

    # Print shapes for debugging
    #print('\n')
    #print(f"Shape of logits in compute_metrics: {predictions.shape}")
    #print(f"Shape of labels in compute_metrics: {labels.shape}")
    #('\n')


    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    em = metric.compute(predictions=decoded_preds, references=decoded_labels)["exact_match"]

    return {"exact_match": em}



#9. Training Arguments and Trainer
training_args = TrainingArguments(
    output_dir ="/content/gdrive/MyDrive/model/GNNT2SQL",
    logging_dir="/content/gdrive/MyDrive/model/GNN-T2SQL/logs",

    per_device_train_batch_size=1,  # Slightly increased, but be cautious
    gradient_accumulation_steps=4,  # Adjusted for effective batch size of 4

    # Number of epochs and early stopping
    num_train_epochs=1,  # Start with a few epochs and monitor validation loss
    #early_stopping_patience=3,  # Enable early stopping to prevent overfitting

    # Learning rate and scheduler
    learning_rate=5e-5,  # A reasonable starting point, adjust if needed
    lr_scheduler_type="linear",  # Or try other schedulers like "cosine"
    warmup_steps=500,  # Warmup is crucial, especially with a larger learning rate

    # Evaluation and saving
    eval_strategy="steps",
    eval_steps=50,  # Evaluate less frequently to save time #500
    save_strategy="steps",
    save_steps=50, #500
    logging_strategy="steps",
    logging_steps=100,

    # Other settings
    push_to_hub=False,
    load_best_model_at_end=True,
    use_legacy_prediction_loop=False,
    metric_for_best_model="eval_exact_match",
    report_to="tensorboard",
    #generation_max_length=2048,  # Adjust if needed based on your data
)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)

from torch.nn.utils.rnn import pad_sequence
from torch_geometric.data import Batch as GraphBatch  # Note the import
import torch
import torch_geometric.data
from torch.nn.utils.rnn import pad_sequence


class GraphDataCollatorForSeq2Seq:
    def __init__(self, tokenizer, model=None, label_pad_token_id=-100, pad_to_multiple_of=None):
        self.tokenizer = tokenizer
        self.model = model
        self.label_pad_token_id = label_pad_token_id
        self.pad_to_multiple_of = pad_to_multiple_of

    def __call__(self, features):
        # Separate standard features from graph edges
        # Extract labels before padding and handle potentially empty sequences
        labels = [feature["labels"] if feature["labels"].numel() > 0
                  else torch.tensor([self.label_pad_token_id], dtype=torch.long)
                  for feature in features]

        # Extract sample_ids
        sample_ids = [feature["sample_ids"] for feature in features]

        standard_features = [{k: v for k, v in feature.items() if k != "edges" and k != "labels"} for feature in features]
        edges = [feature["edges"] for feature in features]

        # Collate standard features (input_ids, attention_mask) using default collator
        collated_standard_features =  DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            label_pad_token_id=self.label_pad_token_id,
            pad_to_multiple_of=self.pad_to_multiple_of
        )(standard_features)

        # Pad input_ids and attention_mask
        input_ids = pad_sequence([f['input_ids'] for f in standard_features], batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence([f['attention_mask'] for f in standard_features], batch_first=True, padding_value=0)

         # Pad labels separately
        labels = pad_sequence(labels, batch_first=True, padding_value=self.label_pad_token_id)

        # Create batch for graph data
        graph_data_list = []
        for i in range(len(edges)):
            # Convert to PyTorch Geometric Data
            graph_data_list.append(torch_geometric.data.Data(
                x=collated_standard_features['input_ids'][i].unsqueeze(1),  # Node features (input_ids)
                edge_index=edges[i],                   # Edge index
                # Use num_edges for batch index to ensure correct batching in PyG
                batch=torch.tensor([i] * edges[i].size(1))
            ))
        batched_graph = GraphBatch.from_data_list(graph_data_list)  # Batch graphs

        #print(f"Sample GraphDataCollatorForSeq2Seq ID: {sample_ids}")

         # Include sample_ids in the collated features
        collated_features = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "edges": edges,
            "sample_ids": sample_ids  # Add sample_ids here
        }

        return collated_features


# 10a.  Data Collator
data_collator = GraphDataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)


# 10AA. Training Arguments and Trainer

from transformers import Trainer
from transformers.trainer_utils import EvalLoopOutput, PredictionOutput,  has_length

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

from transformers.trainer_utils import EvalPrediction
from transformers import Trainer, TrainerCallback

from typing import Any, Dict, List, Optional, Tuple, Union
#from transformers.trainer_pt_utils import nested_truncate  # Updated import#

from transformers.trainer_pt_utils import nested_truncate, nested_concat, nested_numpify, nested_detach  # Updated imports


######
from torch.nn import CrossEntropyLoss

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from transformers import Trainer
from transformers.trainer_utils import EvalPrediction
import torch
import torch.nn.functional as F

def estimated_num_samples(dataloader: DataLoader):
    """
    This function will attempt to determine the number of samples in a DataLoader.

    Args:
        dataloader (DataLoader): The DataLoader to estimate the number of samples from.

    Returns:
        int: The estimated number of samples, or 0 if estimation is not possible.
    """
    if hasattr(dataloader, "dataset") and hasattr(dataloader.dataset, "__len__"):
        return len(dataloader.dataset)  # Use dataset length if available
    elif hasattr(dataloader, "batch_sampler") and hasattr(dataloader.batch_sampler, "sampler") and hasattr(dataloader.batch_sampler.sampler, "__len__"):
        return len(dataloader.batch_sampler.sampler)  # Use sampler length if available
    else:
        # If neither is available, return 0
        warnings.warn("Could not estimate the number of samples in the dataloader. Returning 0.")
        return 0

class CustomTrainer(Trainer):
    _id=0

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        self.tokenizer = tokenizer
        inputs = self._prepare_inputs(inputs)
        labels = inputs.pop("labels", None)

        print('\n\n')
        print("**** Prediction Step ****")


        with torch.no_grad():
            outputs = model(**inputs)

            # Access loss and logits based on the type of outputs
            if isinstance(outputs, MyCausalLMOutputWithPast):
                loss = outputs.loss
                logits = outputs.logits
            elif isinstance(outputs, tuple):
                # Check if the tuple has at least two elements before accessing
                if len(outputs) >= 2:
                    loss = outputs[0]  # Assuming loss is the first element
                    logits = outputs[1]  # Assuming logits is the second element
                else:
                    # Handle the case where the tuple has fewer than two elements
                    raise ValueError("Output tuple from model has fewer than two elements")
            else:
                #raise ValueError("Unexpected output type from model")
                # Access loss and logits as attributes for other types
                loss = outputs["loss"] # Access loss from the dictionary
                logits = outputs["logits"] # Access logits from the dictionary


            # Debugging Logging (ensure labels exist before accessing)
            if not prediction_loss_only:
                print('\n\n')
                print("Shape of logits in prediction_step:", logits.shape)
                if labels is not None:  # Only print if labels exist
                    print("Shape of labels in prediction_step:", labels.shape)

            if prediction_loss_only:
                if isinstance(loss, torch.Tensor):
                    loss = loss.mean().detach()
                return (loss, None, None)

            max_new_tokens = 1024 - inputs['input_ids'].shape[1]

            # Modify this part to handle the generated IDs
            if max_new_tokens <= 0:
                print('\n\n')
                print("No new tokens to generate. An increase in the sample size for training is required. ")
                #print("#TOKENS: ",max_new_tokens)
                # Input is already at max length, no need to generate
                generated_ids = inputs['input_ids']
            else:
                generated_ids = model.encoder.generate(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    max_new_tokens=max_new_tokens,
                    num_beams=5
                )

            # Check the shape of generated_ids
            #print("Shape of generated_ids:", generated_ids.shape)



        # Flatten the generated_ids to a 1D list before decoding
        flattened_generated_ids = generated_ids.view(-1).tolist()
        predictions=generated_ids

        # Now decode using the flattened list
        predictions_decoder = self.tokenizer.decode(flattened_generated_ids, skip_special_tokens=True)
        Q = self.tokenizer.decode(inputs['input_ids'].view(-1).tolist(), skip_special_tokens=True)
        A = self.tokenizer.decode(labels.view(-1).tolist(), skip_special_tokens=True)
        #print("Sample IDs in prediction_step:", inputs['sample_ids'][0])


        # Debugging Logging
        #print("Shape of predictions in prediction_step:", predictions.shape)
        if labels is not None:
            print('\n\n')
            # Extract sample_ids
            sample_ids = int(self._id)+1  # Use self._id here
            #print("Sample IDs in prediction_step:", sample_ids)
            print("Sample IDs in prediction_step:", inputs['sample_ids'])

            print("Question:", Q)
            print("Decoded Original Answer BEFORE Predictions:", A)
            print('\n\n')

            print("Decoded Predictions:", predictions_decoder)

            return (loss, predictions, labels)

    def _prediction_loop(self, dataloader: DataLoader, description: str, prediction_loss_only: Optional[bool] = None, ignore_keys: Optional[List[str]] = None, metric_key_prefix: str = "eval") -> Union[Tuple[torch.Tensor, torch.Tensor], EvalPrediction]:


         # In case you have a callback that needs length eventually
        if has_length(dataloader):
            num_samples = len(dataloader.dataset)
        # The dataset does not support __len__, estimate the number of samples.
        else:
            num_samples = estimated_num_samples(dataloader)

        # Number of losses has been rounded to a multiple of batch_size and in a distributed training, the number of
        # samplers has been rounded to a multiple of batch_size, so we truncate.
        if all_losses:
            all_losses = all_losses[:num_samples]
        if all_preds:
            all_preds = nested_truncate(all_preds, num_samples)
        if all_labels:
            all_labels = nested_truncate(all_labels, num_samples)

        # 8.  Compute Metrics and Average Loss
        metrics = self.compute_metrics((all_preds, all_labels))
        average_loss = torch.mean(torch.stack(all_losses))
        metrics[f"{metric_key_prefix}_loss"] = average_loss.item()

        # 9.  Log the Metrics
        self.log(metrics)

        # 10. Return Based on Whether It's a Prediction or Evaluation
        if prediction_loss_only:
            return (metrics, None, None)

        return EvalPrediction(predictions=all_preds, label_ids=all_labels, metrics=metrics)


    def predict(self, test_dataset: Dataset, ignore_keys: Optional[List[str]] = None) -> PredictionOutput:
        test_dataloader = self.get_test_dataloader(test_dataset)
        return self.prediction_loop(test_dataloader, description="Prediction")

    def evaluate(
        self,
        eval_dataset: Optional[Dataset] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ) -> Dict[str, float]:

        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        output = self.prediction_loop(
            eval_dataloader,
            description="Evaluation",
        )

        return output.metrics


######

# 10A. Trainer (Modified)
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)


from transformers import TrainerCallback
class LossLoggingCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 50000 == 0:  # Log every 100 steps (adjust as needed)
            print(f"Step {state.global_step} - Loss: {state.loss}")




# 11. Train the model
# Add the Callback to the Trainer
trainer.add_callback(LossLoggingCallback())

# Add the Early Stopping to the Trainer
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))

trainer.train()


# 12. Evaluate on the test set
#test_results = trainer.evaluate(eval_dataset)
#print('\n\n')
#print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')
#print(f'Test Exact Match. (Evaluate on the test set): {test_results["eval_exact_match"]:.4f}')
#print('\n\n')

1. Units per Hour: You're using 4.82 compute units every hour.

2. Total Time Available:

Divide your total compute units by your hourly usage rate: 309.66 units / 4.82 units/hour = 64.24 hours

## Compute Metrics

What is Logical Correctness?

In the realm of T2SQL (Text-to-SQL), logical correctness goes beyond mere syntactic accuracy and execution success. It evaluates whether the generated SQL query genuinely captures the intent and nuances of the natural language input, ensuring that it retrieves the desired information from the database.

Consider these scenarios where a generated SQL query might be syntactically valid and even execute without errors, yet still be logically incorrect:

Incorrect column or table selection: The query might fetch data from the wrong columns or tables, resulting in irrelevant or inaccurate results.
Incorrect filtering or aggregation: The WHERE clause or aggregation functions (e.g., SUM, COUNT) might be misapplied, leading to filtered or aggregated data that doesn't align with the user's intent.
Incorrect joins: If the natural language input implies relationships between multiple tables, the generated query might have incorrect or missing joins, producing misleading results.
Subtle semantic mismatches: Even if the query produces results, they might not fully capture the nuances and implied meaning of the original question or instruction.
Why is it Important?

While execution accuracy is crucial, logical correctness ensures that the generated SQL truly "understands" the user's request and provides the right answer, not just an answer. It's a key indicator of the T2SQL model's ability to reason about the data schema and the user's intent.

How to Measure Logical Correctness

Measuring logical correctness can be challenging, as it often requires a deeper understanding of the underlying data schema and the subtle nuances of natural language. Here are some common approaches:

Manual Inspection:

A human evaluator examines a sample of generated queries and their corresponding ground truth queries to assess if they capture the same intent.
Pros: Provides valuable qualitative insights and can catch subtle semantic mismatches.
Cons: Time-consuming and not scalable for large datasets.
SQL Parsing and Comparison:

Use a SQL parser to extract structural components (e.g., SELECT, FROM, WHERE, GROUP BY) from both generated and ground truth queries.
Compare these components to identify mismatches or inconsistencies.
Pros: Can be automated and is relatively scalable.
Cons: Might miss subtle semantic differences or struggle with complex SQL constructs.
Result Comparison:

Execute both the generated and ground truth queries against the database.
Compare the results to see if they are equivalent or sufficiently similar.
Pros: Directly measures the impact of logical errors on the output.
Cons: Requires access to the database, can be computationally expensive, and might not capture all types of logical errors.
Hybrid Approaches:

Combine manual inspection with automated techniques to leverage their strengths.
For example, use SQL parsing to identify potential logical errors and then have a human evaluator review those cases.
Choosing the Right Approach

The ideal approach for measuring logical correctness depends on factors like:

Data schema complexity: More complex schemas might require more sophisticated techniques.
Evaluation scale: Manual inspection might be feasible for smaller datasets, while automated methods are necessary for larger ones.
Available resources: Access to a database and computational power will influence the feasibility of certain approaches.
Desired level of rigor: The trade-off between efficiency and thoroughness will guide your choice.
Remember, evaluating logical correctness is an ongoing challenge in T2SQL research. Combining multiple approaches and iteratively refining your evaluation metrics will lead to more robust and reliable T2SQL models.


In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
from datasets import load_metric
import sqlparse
import psycopg2

# Assuming you have 'tokenizer' and 'sentence_transformer_model' defined elsewhere

# Load the metric for exact match calculation
metric = load_metric("exact_match")

def compute_metrics(eval_pred, db_config=None):
    all_preds, all_labels = eval_pred

    # Convert predictions and labels to tensors
    predictions = [
        torch.tensor(pred) if not isinstance(pred, torch.Tensor) else pred
        for pred in all_preds
    ]
    labels = [
        torch.tensor(label) if not isinstance(label, torch.Tensor) else label
        for label in all_labels
    ]

    # Filter out None values
    predictions = [pred for pred in predictions if pred is not None]
    labels = [label for label in labels if label is not None]

    # Stack predictions and labels if they exist
    if predictions:
        predictions = torch.stack(predictions).squeeze()
    else:
        predictions = torch.tensor([])
    if labels:
        labels = torch.stack(labels).squeeze()
    else:
        labels = torch.tensor([])

    # Handle single prediction/label cases
    if predictions.dim() == 0:
        predictions = predictions.unsqueeze(0)
    if labels.dim() == 0:
        labels = labels.unsqueeze(0)

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute exact match accuracy
    em = metric.compute(predictions=decoded_preds, references=decoded_labels)["exact_match"]

    # Compute semantic similarity
    embeddings_pred = sentence_transformer_model.encode(decoded_preds)
    embeddings_labels = sentence_transformer_model.encode(decoded_labels)
    semantic_similarity = util.cos_sim(embeddings_pred, embeddings_labels).mean()

    # Execution Accuracy (if db_config is provided)
    if db_config:
        execution_accuracy_scores = []
        with psycopg2.connect(**db_config) as conn:
            with conn.cursor() as cur:
                for pred_sql, label_sql in zip(decoded_preds, decoded_labels):
                    try:
                        # Execute predicted query
                        cur.execute(pred_sql)
                        pred_results = cur.fetchall()

                        # Execute label (ground truth) query
                        cur.execute(label_sql)
                        label_results = cur.fetchall()

                        # Compare results
                        if pred_results == label_results:
                            execution_accuracy_scores.append(1)
                        else:
                            execution_accuracy_scores.append(0)

                            # Optional: Log mismatches for debugging
                            logging.debug(f"Mismatch: Predicted: {pred_results}, Label: {label_results}")

                    except psycopg2.Error as e:
                        execution_accuracy_scores.append(0)
                        logging.error(f"Error executing SQL: {e}")

        execution_accuracy = sum(execution_accuracy_scores) / len(execution_accuracy_scores)
    else:
        execution_accuracy = 0.0

    # Logical Correctness (SQL Parsing and Comparison)
    logical_correctness_scores = []
    for pred_sql, label_sql in zip(decoded_preds, decoded_labels):
        parsed_pred = sqlparse.parse(pred_sql)[0]
        parsed_label = sqlparse.parse(label_sql)[0]

        pred_components = {
            "select": [token.value for token in parsed_pred.tokens if isinstance(token, sqlparse.sql.IdentifierList)],
            "from": [token.value for token in parsed_pred.tokens if isinstance(token, sqlparse.sql.Identifier)],
            # ... extract other components like WHERE, GROUP BY, etc.
        }
        label_components = {
            "select": [token.value for token in parsed_label.tokens if isinstance(token, sqlparse.sql.IdentifierList)],
            "from": [token.value for token in parsed_label.tokens if isinstance(token, sqlparse.sql.Identifier)],
            # ... extract other components like WHERE, GROUP BY, etc., using the same logic as for pred_components
        }

        score = 0
        for component_type in ["select", "from", ...]:
            if pred_components[component_type] == label_components[component_type]:
                score += 1
            elif set(pred_components[component_type]) == set(label_components[component_type]):
                score += 0.5
            # ... add more sophisticated comparison logic if needed

        logical_correctness_scores.append(score)

    logical_correctness = sum(logical_correctness_scores) / len(logical_correctness_scores)


    return {
        "exact_match": em,
        "semantic_similarity": semantic_similarity.item(),
        "execution_accuracy": execution_accuracy,
        "logical_correctness": logical_correctness,
    }


## MISTRAL-MODEL

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from trl import setup_chat_format

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model and tokenizer
mistral_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

PackageNotFoundError: No package metadata was found for bitsandbytes

## Postgresql Setup

In [4]:
#ADDED By FM 01/06/2024
!apt-get update -y
!apt-get install postgresql-14 -y

!service postgresql restart
!sudo apt install postgresql-server-dev-all

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ign:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,326 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:14 https://r2u.stat.illinois.edu

In [5]:
!sudo -u postgres psql -c "CREATE USER postgres WITH SUPERUSER"
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

#ERROR:  role "postgres" already exists
#ALTER ROLE

QUERY_create='CREATE TABLE table_name_24 (score VARCHAR, date VARCHAR)'


QUERY_select='SELECT 2009 FROM table_name_50 WHERE 2011 = "a"'

ERROR:  role "postgres" already exists
ALTER ROLE


In [6]:
def table_creator(query):
    import os
    import psycopg2 as ps
    import pandas as pd

    DB_NAME = "postgres"
    DB_USER = "postgres"
    DB_PASS = "postgres"
    DB_HOST = "localhost"
    DB_PORT = "5432"

    conn = ps.connect(database=DB_NAME,
                  user=DB_USER,
                  password=DB_PASS,
                  host=DB_HOST,
                  port=DB_PORT)

    cur = conn.cursor() # creating a cursor

    # Wrap the execute command in a try-except block to handle potential errors
    try:
        cur.execute("""
                            %s
                            """%query)
        conn.commit()
        print("Table Created successfully")
    except Exception as e:
        conn.rollback() # Rollback the transaction in case of an error
        print("Error creating table:", e)

    conn.close()

In [7]:
import os
import psycopg2 as ps
import pandas as pd

DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASS = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

In [8]:
import os
import psycopg2 as ps
import pandas as pd

def table_select(query):
    conn = ps.connect(database=DB_NAME,
                      user=DB_USER,
                      password=DB_PASS,
                      host=DB_HOST,
                      port=DB_PORT)
    print("Database connected successfully")

    #query = query.replace('"', "'") # Replace double quotes with single quotes for potential date values

    try:

        #df = pd.read_sql_query("%s"%query, con=conn)
        #print('rec: %'%df) # Print the resulting DataFrame

        cur = conn.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        conn.commit()
        conn.close()
        print('\n')
        print('Record(s): %s \n'%len(rows))
        for row in rows:
            print(row)


        eqc=1

    except Exception as e:
        eqc=0
        #conn.rollback() # Rollback the transaction in case of an error
        print("Error executing query:", e)
        #print('TABLE IS EMPTY')
        conn.commit()
        conn.close()

    return eqc

In [9]:
table_creator(QUERY_create)

Table Created successfully


## evaluate_model

In [14]:
import logging
import psycopg2
from transformers import Trainer

# Your PostgreSQL configuration
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASS = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

def evaluate_model(model, tokenizer, sentence_transformer_model, eval_dataset):
    """
    Evaluates a model on a given dataset and computes exact match and semantic similarity metrics,
    after creating necessary tables in the PostgreSQL database.

    Args:
        model: The fine-tuned model to be evaluated.
        tokenizer: The tokenizer used for encoding and decoding text.
        sentence_transformer_model: The sentence transformer model used for semantic similarity.
        eval_dataset: The dataset on which the model will be evaluated.

    Returns:
        A dictionary containing the evaluation results: 'exact_match' and 'semantic_similarity'.
    """

    # 1. Table Creation
    db_config = {
        'database': DB_NAME,
        'user': DB_USER,
        'password': DB_PASS,
        'host': DB_HOST,
        'port': DB_PORT
    }

    with psycopg2.connect(**db_config) as conn:
        with conn.cursor() as cur:
            for example in eval_dataset.dataset:
                context = example['context']
                create_table_statements = context.split(';')
                for create_table_statement in create_table_statements:
                    if create_table_statement.strip():
                        try:
                            cur.execute(create_table_statement)
                            conn.commit()
                        except psycopg2.Error as e:
                            logging.error(f"Error creating table: {e}")

    # Create a Trainer instance with the compute_metrics function
    trainer = Trainer(
        model=model,
        compute_metrics=compute_metrics,  # Remove the lambda function and db_config
    )

    # Evaluate the model
    results = trainer.evaluate(eval_dataset)

    return results


## EVALUATION-TRAINER

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset

from peft import LoraConfig, get_peft_model, TaskType

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)

#from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import spacy
import numpy as np

from torch_geometric.nn import GAT

from trl import setup_chat_format

import colab_env
import evaluate

In [28]:
# Assuming 'edges' data is present in the eval_dataset, modify the TextToSQLDataset class to include 'edges' in the returned dictionary.
class TextToSQLDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset[idx]
        # Assuming 'edges' is a key in your dataset
        edges = row.get('edges')
        encoding = self.tokenizer(
            row["question"],
            row["context"],
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt",
        )
        # Include 'edges' in the returned dictionary
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": self.tokenizer(row["answer"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")["input_ids"].squeeze(),
            "edges": edges # Make sure edges are properly formatted for your GraphModel
        }

In [29]:
def compute_metrics(eval_pred):
    all_preds, all_labels = eval_pred

    # Convert all elements to tensors, handling different data types
    predictions = [torch.tensor(pred) if not isinstance(pred, torch.Tensor) else pred for pred in all_preds]
    labels = [torch.tensor(label) if not isinstance(label, torch.Tensor) else label for label in all_labels]

    # Filter out any None values before stacking
    predictions = [pred for pred in predictions if pred is not None]
    labels = [label for label in labels if label is not None]

    # Convert to tensors and stack (only if there are predictions/labels)
    if predictions:
        predictions = torch.stack(predictions).squeeze()
    else:
        predictions = torch.tensor([])  # Empty tensor if no predictions

    if labels:
        labels = torch.stack(labels).squeeze()
    else:
        labels = torch.tensor([])  # Empty tensor if no labels

    # Handle cases where only one prediction/label is present (avoid squeezing to a scalar)
    if predictions.dim() == 0:
        predictions = predictions.unsqueeze(0)
    if labels.dim() == 0:
        labels = labels.unsqueeze(0)

    # Print shapes for debugging
    print('\n')
    print(f"Shape of logits in compute_metrics: {predictions.shape}")
    print(f"Shape of labels in compute_metrics: {labels.shape}")
    print('\n')

    # Ensure predictions is a list of lists
    predictions = predictions.tolist()
    # Remove the extra nesting if present
    if isinstance(predictions[0], list):
        if isinstance(predictions[0][0], list):
            predictions = [p[0] for p in predictions]

    # Convert logits to predicted token ids
    predictions = torch.tensor(predictions).argmax(dim=-1)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Load the metric
    metric = evaluate.load("exact_match")

    em = metric.compute(predictions=decoded_preds, references=decoded_labels)["exact_match"]

    return {"exact_match": em}

In [ ]:
from transformers import AutoTokenizer
from peft import PeftModel # PeftModel is now correctly imported from peft
from datasets import load_dataset
import evaluate
from sentence_transformers import SentenceTransformer
import numpy as np

# Load your fine-tuned model
output_dir = "/content/gdrive/MyDrive/model/GNNT2SQL"

#model_id ='/content/gdrive/MyDrive/model/GNNT2SQL/checkpoint-100/'

model_id = '/content/gdrive/MyDrive/model/GNNT2SQL/checkpoint-1950/'

# Load the base model first
#model = AutoModelForCausalLM.from_pretrained("google/flan-t5-xl")


# Use PeftModel to load the model, pass the model object and model_id as arguments
model = PeftModel.from_pretrained(mistral_model, model_id)


# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set padding side to 'left'
tokenizer.padding_side = 'left'

# Add the padding token
tokenizer.pad_token = tokenizer.eos_token

# Load the sentence transformer model
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

# Prepare your evaluation dataset
eval_dataset = load_dataset("b-mc2/sql-create-context")["train"].shuffle(seed=42)

POC_valsample = 100
eval_dataset = eval_dataset.select(np.random.choice(len(eval_dataset), POC_valsample, replace=False))

# Assuming TextToSQLDataset is a custom class, ensure it handles 'edges' data
eval_dataset = TextToSQLDataset(eval_dataset, tokenizer)

# Evaluate the model
results = evaluate_model(model, tokenizer, sentence_transformer_model, eval_dataset)

# Print the results
print(f'Exact Match: {results["eval_exact_match"]:.4f}')
#print(f'Semantic Similarity: {results["eval_semantic_similarity"]:.4f}')

# Ensure that the 'edges' data is correctly included and passed to the model within the TextToSQLDataset class and evaluate_model function.

In [ ]:
def compute_metrics(eval_pred):
    all_preds, all_labels = eval_pred

    # Convert all elements to tensors, handling different data types
    predictions = [torch.tensor(pred) if not isinstance(pred, torch.Tensor) else pred for pred in all_preds]
    labels = [torch.tensor(label) if not isinstance(label, torch.Tensor) else label for label in all_labels]

    # Filter out any None values before stacking
    predictions = [pred for pred in predictions if pred is not None]
    labels = [label for label in labels if label is not None]

    # Convert to tensors and stack (only if there are predictions/labels)
    if predictions:
        predictions = torch.stack(predictions).squeeze()
    else:
        predictions = torch.tensor([])  # Empty tensor if no predictions

    if labels:
        labels = torch.stack(labels).squeeze()
    else:
        labels = torch.tensor([])  # Empty tensor if no labels

    # Handle cases where only one prediction/label is present (avoid squeezing to a scalar)
    if predictions.dim() == 0:
        predictions = predictions.unsqueeze(0)
    if labels.dim() == 0:
        labels = labels.unsqueeze(0)

    # Print shapes for debugging
    print('\n')
    print(f"Shape of logits in compute_metrics: {predictions.shape}")
    print(f"Shape of labels in compute_metrics: {labels.shape}")
    print('\n')

    # Ensure predictions is a list of lists
    predictions = predictions.tolist()
    # Remove the extra nesting if present
    if isinstance(predictions[0], list):
        if isinstance(predictions[0][0], list):
            predictions = [p[0] for p in predictions]

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    em = metric.compute(predictions=decoded_preds, references=decoded_labels)["exact_match"]

    return {"exact_match": em}

## EVALUATION STAND-ALONE


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig, Trainer, TrainingArguments,
    set_seed,
    EarlyStoppingCallback,
    DataCollatorForSeq2Seq
)

from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel,
    PeftConfig,
    PrefixTuningConfig,
    PromptEncoderConfig,
    TaskType
)


from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from torch_geometric.nn import GAT
import spacy
import numpy as np
import torch.nn as nn
import evaluate


# Suppress warnings
import warnings
warnings.simplefilter('ignore')

warnings.filterwarnings("ignore", message="The installed version of bitsandbytes was compiled without GPU support.")


# Load spaCy English model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    # Download if not already downloaded
    spacy.cli.download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")


In [26]:
#  GraphModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv

import torch
import torch.nn as nn
from torch_geometric.nn import GATv2Conv


#from transformers import CausalLMOutputWithPast

from dataclasses import dataclass
from typing import Optional, Tuple

@dataclass
class MyCausalLMOutputWithPast:
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


class GraphModel(nn.Module):
    def __init__(self, encoder, tokenizer):
        super(GraphModel, self).__init__()
        self.encoder = encoder
        self.config = encoder.model.config
        self.gatv2 = GATv2Conv(
            in_channels=self.config.hidden_size,
            out_channels=self.config.hidden_size,
            heads=8,
            concat=False,
        )

        # Max Pooling
        self.pool = lambda x, batch: torch.max(x, dim=0, keepdim=True)[0]

        # Additional Feedforward Layer
        self.ffn = nn.Sequential(
            nn.Linear(self.config.hidden_size, self.config.hidden_size * 2),
            nn.ReLU(),
            nn.Linear(self.config.hidden_size * 2, self.config.hidden_size),
        )

        self.lm_head = nn.Linear(self.config.hidden_size, self.config.vocab_size)
        self.tokenizer = tokenizer

        # Add generation config (you might need to adjust this based on your specific needs)
        self.generation_config = encoder.generation_config

    # Forward Pass
    #def forward(self, input_ids, attention_mask, edges, labels=None, inputs_embeds=None, sample_ids=None, output_attentions=False, output_hidden_states=False, return_dict=False):

    def forward(self, input_ids, attention_mask, edges, labels=None, inputs_embeds=None, sample_ids=None,
                output_attentions=None, output_hidden_states=None, return_dict=None):




        # Print vocabulary sizes
        #print('\n')
        #print("Mistral Model Vocab Size:", self.encoder.config.vocab_size)
        #print("Tokenizer Vocab Size:", self.tokenizer.vocab_size)
        #print('\n')


        # 1. Token Embeddings (Encoder)
        if input_ids is not None:
            encoder_outputs = self.encoder(
                input_ids.to(self.encoder.model.device),
                attention_mask=attention_mask.to(self.encoder.model.device),
                #output_hidden_states=True
            )
            embeddings = encoder_outputs.hidden_states[-1]
            print("Encoder output shape:", embeddings.shape)  # Inspect encoder output

        elif inputs_embeds is not None:
            embeddings = inputs_embeds
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        # Ensure correct shape for GATv2Conv
        if embeddings.dim() > 2:
            embeddings = embeddings.view(-1, embeddings.shape[-1])

        # 2. Edge Index Creation (Batched, with Enhanced Error Handling)
        edge_index = []
        node_offset = 0
        for i, graph_edges in enumerate(edges):
            if graph_edges is None or graph_edges.numel() == 0:  # Check if graph_edges is None or empty
                num_nodes = input_ids.size(1)
                # Create self-loops for isolated nodes if no edges are provided
                graph_edges = torch.arange(node_offset, node_offset + num_nodes, device=embeddings.device)
                graph_edges = graph_edges.repeat(2, 1) # Repeat the tensor, not the arange object
            else: # Add this else block to handle the case when edges are present
                # Ensure graph_edges is a tensor before adding offset
                if not isinstance(graph_edges, torch.Tensor):
                    graph_edges = torch.tensor(graph_edges, dtype=torch.long, device=embeddings.device)
                graph_edges += node_offset  # Now safe to add offset
            edge_index.append(graph_edges)
            node_offset += input_ids.size(1)
        edge_index = torch.cat(edge_index, dim=1)

        # 3. GATv2 Layer
        graph_out = self.gatv2(embeddings, edge_index)

        # 3. GATv2 Layer
        graph_out = self.gatv2(embeddings, edge_index)
        print("GATv2 output shape:", graph_out.shape)  # Inspect GATv2 output

        # Print logits before and after applying logits processor
        print("Original logits:", logits)
        filtered_logits = logits_processor(None, logits)
        print("Filtered logits:", filtered_logits)


        # 4. Pooling (using max pooling)
        batch = torch.arange(len(edges), device=graph_out.device).repeat_interleave(input_ids.size(1))
        pooled = self.pool(graph_out, batch).unsqueeze(1)

        # Additional Feedforward Layer
        pooled = self.ffn(pooled)

        # 5. LM Head
        logits = self.lm_head(pooled)

        from torch.nn import CrossEntropyLoss
        # 6. Loss Calculation (if labels provided)
        loss = None
        if labels is not None:

             # Print Sample IDs first
            print(f"\nIteration/Step: {trainer.state.global_step}")
            mask = (labels != -100).float()

            #for i, sample_id in enumerate(sample_ids):
            #    print(f"Sample ID: {sample_id.item()}")


            # Now decode and print input, target, and loss for each sample
            with torch.no_grad():
                input_text = self.tokenizer.batch_decode(input_ids, skip_special_tokens=True)
                target_text = self.tokenizer.batch_decode(labels, skip_special_tokens=True)
                for i, sample_id in enumerate(sample_ids):
                    print(f"Sample ID: {sample_id.item()}")
                    print("Decoded Input:", input_text[i])
                    print("Decoded Target (Labels):", target_text[i])



            # Apply softmax to logits to get probabilities
            log_probs = F.log_softmax(logits, dim=-1)

            # Calculate loss
            #loss_fct = nn.NLLLoss(ignore_index=-100)  # Use NLLLoss for log probabilities

            loss_fct = CrossEntropyLoss(ignore_index=-1)

            # Reshape log_probs to match target shape (remove the extra dimension)
            log_probs = log_probs.squeeze(1)


            #labels = labels[:, 0] ### WORK UNTIL BEFORE THE LAST INTERACTION
            #labels = labels.squeeze(1)   # Remove extra dimension from labels ## DON'T work
            #labels = labels.view(-1)  # Flatten labels to a 1D tensor DON'T work

            labels = labels[:, 0]

            # Calculate loss with the correctly shaped labels
            #loss = loss_fct(log_probs, labels[:, 0])  # Access the first element of each label sequence


            loss = loss_fct(log_probs, labels)



            loss_per_sample = (loss * mask).sum(dim=1) / mask.sum(dim=1)
            print(f"Loss per sample: {loss_per_sample.item()}")


        # 7. Return (Modified to return a tuple)
            if labels is not None:
                return (loss, logits, None)
            else:
                return (None, logits, None)

        return MyCausalLMOutputWithPast(
                loss=loss,
                logits=logits,
                past_key_values=encoder_outputs.past_key_values,
                hidden_states=encoder_outputs.hidden_states,
                attentions=encoder_outputs.attentions,
        )


    def prepare_inputs_for_generation(self, input_ids, edges, attention_mask=None, **kwargs):
        if isinstance(self, PeftModel):
            return self.base_model.prepare_inputs_for_generation(input_ids, edges, attention_mask, **kwargs)

        batch_size = input_ids.size(0)
        if batch_size > 1:
            batched_edges = []
            node_offset = 0
            for i in range(batch_size):
                graph_edges = edges[i]
                batched_edges.extend([(src + node_offset, dst + node_offset) for src, dst in graph_edges])
                node_offset += input_ids.size(1)
            edge_index = torch.tensor(batched_edges, dtype=torch.long).t().contiguous().to(input_ids.device)
        else:
            edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous().to(input_ids.device)

        model_inputs = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "edges": edge_index,
            "past_key_values": kwargs.get("past_key_values", None),
        }
        return model_inputs

# END GraphModel

In [3]:
#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)


from torch.nn.utils.rnn import pad_sequence
from torch_geometric.data import Batch as GraphBatch  # Note the import
import torch
import torch_geometric.data
from torch.nn.utils.rnn import pad_sequence

class GraphDataCollatorForSeq2Seq:
    def __init__(self, tokenizer, model=None, label_pad_token_id=-100, pad_to_multiple_of=None):
        self.tokenizer = tokenizer
        self.model = model
        self.label_pad_token_id = label_pad_token_id
        self.pad_to_multiple_of = pad_to_multiple_of

    def __call__(self, features):
        # Separate standard features from graph edges
        # Extract labels before padding and handle potentially empty sequences
        labels = [feature["labels"] if feature["labels"].numel() > 0
                  else torch.tensor([self.label_pad_token_id], dtype=torch.long)
                  for feature in features]

        # Extract sample_ids
        sample_ids = [feature["sample_ids"] for feature in features]

        standard_features = [{k: v for k, v in feature.items() if k != "edges" and k != "labels"} for feature in features]
        edges = [feature["edges"] for feature in features]

        # Collate standard features (input_ids, attention_mask) using default collator
        collated_standard_features =  DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            label_pad_token_id=self.label_pad_token_id,
            pad_to_multiple_of=self.pad_to_multiple_of
        )(standard_features)

        # Pad input_ids and attention_mask
        input_ids = pad_sequence([f['input_ids'] for f in standard_features], batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence([f['attention_mask'] for f in standard_features], batch_first=True, padding_value=0)

         # Pad labels separately
        labels = pad_sequence(labels, batch_first=True, padding_value=self.label_pad_token_id)

        # Create batch for graph data
        graph_data_list = []
        for i in range(len(edges)):
            # Convert to PyTorch Geometric Data
            graph_data_list.append(torch_geometric.data.Data(
                x=collated_standard_features['input_ids'][i].unsqueeze(1),  # Node features (input_ids)
                edge_index=edges[i],                   # Edge index
                # Use num_edges for batch index to ensure correct batching in PyG
                batch=torch.tensor([i] * edges[i].size(1))
            ))
        batched_graph = GraphBatch.from_data_list(graph_data_list)  # Batch graphs

        #sample_ids = torch.arange(len(edges)).unsqueeze(1)
        #sample_ids = [f["sample_id"] for f in  collated_standard_features]
        #sample_ids = [f["sample_id"] for f in features]

        #print(f"Sample GraphDataCollatorForSeq2Seq ID: {sample_ids}")



         # Include sample_ids in the collated features
        collated_features = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "edges": edges,
            "sample_ids": sample_ids  # Add sample_ids here
        }

        return collated_features


In [4]:
# 2. Mistral Model and Tokenizer
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Model Configuration
config = AutoModelForCausalLM.from_pretrained(model_id).config
config.output_hidden_states = True
config.use_cache = False
#config.torch_dtype = torch.float32
config.torch_dtype = torch.bfloat16

# Load Model with Quantization
mistral_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="flash_attention_2",  # Optimization
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    config=config
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
class TextToSQLDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]


        text = item['question']
        target_text = item['answer']


        # 1. Tokenization (with padding and truncation)
        tokenized_input = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=1024,  # Increase as needed
            return_tensors="pt"
        )
        tokenized_target = self.tokenizer(
            target_text,
            truncation=True,
            padding="max_length",
            max_length=1024,  # Increase as needed
            return_tensors="pt"
        )

        # Flatten lists (if needed)
        tokenized_input = {k: v.squeeze(0) for k, v in tokenized_input.items()}
        tokenized_target = {k: v.squeeze(0) for k, v in tokenized_target.items()}


        # Print input text and target text for debugging
        #print('\n')
        #print(f"Sample ID: {idx}")
        #print(f"  - Text: {text}")
        #print(f"  - Target Text: {target_text}")
        #print('\n')

        sample_ids = torch.tensor([idx])  # Create a tensor with the sample ID

        # 2. Dependency Parsing for Edge Extraction
        doc = nlp(text)
        edges = []
        for token in doc:
            head_i = token.head.i
            if 0 <= head_i < len(doc) and token.dep_ != "ROOT" and token.i != head_i:
                edges.append([token.i, head_i])

        # Edge Index Extraction and Validation
        edges = item.get("edges", edges)  # If "edges" is already present in data, use that

        if not edges:  # Handle empty graphs
            num_nodes = len(tokenized_input["input_ids"])
            edges = [[i, i] for i in range(num_nodes)]  # Self-loops for isolated nodes
        else:
            max_index = len(tokenized_input["input_ids"]) - 1
            edges = [(src, tgt) for src, tgt in edges if 0 <= src <= max_index and 0 <= tgt <= max_index]

        # Create edge index tensor
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        # Convert everything to tensors BEFORE padding/truncation
        input_ids = tokenized_input["input_ids"].clone().detach()
        attention_mask = tokenized_input["attention_mask"].clone().detach()
        labels = tokenized_target["input_ids"].clone().detach()

        # Handle potentially empty target sequences
        if len(labels) == 0:
            labels = torch.tensor([self.tokenizer.pad_token_id], dtype=torch.long)  # Create a single-element tensor with pad token

        # Padding and Truncation for consistent input shapes
        max_length = 1024  # Adjust if needed

        # Ensure that ALL tensors are truncated/padded to the SAME max_length
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]

        # Add padding if necessary
        if len(input_ids) < max_length:
            pad_length = max_length - len(input_ids)
            pad_tensor = torch.full((pad_length,), self.tokenizer.pad_token_id)
            input_ids = torch.cat((input_ids, pad_tensor))
            attention_mask = torch.cat((attention_mask, torch.zeros(pad_length, dtype=torch.long)))

        if len(labels) < max_length:
            pad_length = max_length - len(labels)
            labels = torch.cat((labels, torch.full((pad_length,), -100)))  # Pad labels with -100

        # (Optional) Print statements for debugging
        #print("\n")
        #print("Original text:", text)
        #print("Target text:", target_text)
        #print("Tokenized input IDs:", tokenized_input["input_ids"])
        #print("Tokenized target IDs:", tokenized_target["input_ids"])
        #print("Labels:", labels)
        #print("Edge index:", edge_index)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "edges": edge_index,
            "sample_ids": torch.tensor([idx]),  # Add sample_id here
        }

In [6]:
#1. Load and Prepare Data
dataset = load_dataset("b-mc2/sql-create-context")["train"].shuffle(seed=42)

# Manually define splits
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

eval_dataset = dataset.select(range(train_size, train_size + val_size))
test_dataset = dataset.select(range(train_size + val_size, len(dataset)))


POC_valsample=2
eval_dataset = eval_dataset.select(np.random.choice(len(eval_dataset), POC_valsample, replace=False))
test_dataset = test_dataset.select(np.random.choice(len(test_dataset), POC_valsample, replace=False))



eval_dataset = TextToSQLDataset(eval_dataset, tokenizer)
test_dataset = TextToSQLDataset(test_dataset, tokenizer)

In [7]:
eval_dataset = dataset.select(range(train_size, train_size + val_size))
eval_dataset = TextToSQLDataset(eval_dataset, tokenizer)
eval_dataset[0]

{'input_ids': tensor([    2,     2,     2,  ..., 29538, 29550, 29572]),
 'attention_mask': tensor([0, 0, 0,  ..., 1, 1, 1]),
 'labels': tensor([    2,     2,     2,  ..., 29550, 29538, 29550]),
 'edges': tensor([[ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
         [ 1,  5,  5,  5,  1,  8,  8,  5,  8,  8, 12,  8, 15, 15, 12, 12]]),
 'sample_ids': tensor([0])}

## ORIGINAL - evaluate_model

In [21]:
import torch
from tqdm import tqdm
import evaluate
from peft import PeftModel

def evaluate_model(model_path, eval_dataset, device=None, edges=None):

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = GraphModel(mistral_model, tokenizer)
    #print(model)

    model = PeftModel.from_pretrained(model, model_path, is_trainable=False)
    #print('\n')
    #print(model)
    #print('\n')

    model.to(device)
    model.float()

    metric = evaluate.load("exact_match")
    all_preds, all_labels = [], []

    model.eval()
    model.encoder.eval()


    with torch.no_grad():
        for i in tqdm(range(len(eval_dataset)), desc="Evaluating"):
            data = eval_dataset[i]

            Q = data['input_ids']
            A = data['labels']

            print('\n\n')
            print(f"Sample ID: {i}")
            print(f"  - Text: {tokenizer.decode(Q, skip_special_tokens=True)}")
            print(f"  - Target Text: {tokenizer.decode(A, skip_special_tokens=True)}")
            print('\n')


            # Prepare data for the model
            input_data = {
                "input_ids": Q.unsqueeze(0).to(device).long(),
                "attention_mask": data['attention_mask'].unsqueeze(0).to(device).float()
            }

            if edges is None:
                edges = torch.tensor([[]]).to(device)


            # Model inference
            model_output = model(**input_data, edges=edges.float())
            logits = model_output.logits

            if logits.dim() > 2:
                logits = logits.squeeze(1)

            # Extract predicted tokens
            predicted_tokens = logits.argmax(dim=-1).cpu().tolist()


            # Print shapes and values for debugging
            print("\n\n")
            #print("Model Output:", model_output)
            # Print logits length for debugging
            print("Logits length:", logits.shape[1])
            print("Logits shape:", logits.shape)
            print("Logits dtype:", logits.dtype)
            print("\n")
            print("Predicted IDs:", logits.argmax(dim=-1))
            print("Labels:", data['labels'])
            print("\n")

            # Decode predictions and labels for the current sample
            decoded_pred = tokenizer.decode(predicted_tokens, skip_special_tokens=True) # Decode for single sample
            decoded_label = tokenizer.decode(A.cpu().tolist(), skip_special_tokens=True) # Decode for single sample

            # Append the decoded prediction and label for the current sample
            all_preds.append(decoded_pred)  # Append decoded prediction for current sample
            all_labels.append(decoded_label)  # Append decoded label for current sample

    # Calculate exact match
    em = metric.compute(predictions=all_preds, references=all_labels)["exact_match"]


    print("\nEvaluation Results:")
    print(f"  Exact Match: {em:.4f}")

    return {"exact_match": em}

## WORKING

In [20]:
import torch
from tqdm import tqdm
import evaluate
from peft import PeftModel

def evaluate_model(model_path, eval_dataset, device=None, edges=None):

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = GraphModel(mistral_model, tokenizer)
    model = PeftModel.from_pretrained(model, model_path, is_trainable=False)

    model.to(device)
    model.float()

    metric = evaluate.load("exact_match")
    all_preds, all_labels = [], []

    model.eval()
    model.encoder.eval()

    with torch.no_grad():
        for i in tqdm(range(len(eval_dataset)), desc="Evaluating"):
            data = eval_dataset[i]

            Q = data['input_ids']
            A = data['labels']

            print('\n\n')
            print(f"Sample ID: {i}")
            print(f"  - Text: {tokenizer.decode(Q, skip_special_tokens=True)}")
            print(f"  - Target Text: {tokenizer.decode(A, skip_special_tokens=True)}")
            print('\n')

            # Prepare data for the model
            input_data = {
                "input_ids": Q.unsqueeze(0).to(device).long(),
                "attention_mask": data['attention_mask'].unsqueeze(0).to(device).float()
            }

            if edges is None:
                edges = torch.tensor([[]]).to(device)

            try:
                # Model inference
                model_output = model(**input_data, edges=edges.float())
                logits = model_output.logits

                # Check for NaN or Inf values in logits
                if torch.isnan(logits).any() or torch.isinf(logits).any():
                    print("WARNING: NaN or Inf values detected in logits!")

                if logits.dim() > 2:
                    logits = logits.squeeze(1)
                elif logits.dim() == 1:
                    logits = logits.unsqueeze(0)  # Add a batch dimension if needed

                # Extract predicted tokens
                predicted_tokens = logits.argmax(dim=-1).cpu().tolist()

                # Print shapes and values for debugging
                print("\n\n")
                print("Logits length:", logits.shape[1])
                print("Logits shape:", logits.shape)
                print("Logits dtype:", logits.dtype)
                print("\n")
                print("Predicted IDs:", logits.argmax(dim=-1))
                print("Labels:", data['labels'])
                print("\n")

                # Decode predictions and labels for the current sample
                decoded_pred = tokenizer.decode(predicted_tokens, skip_special_tokens=True)
                decoded_label = tokenizer.decode(A.cpu().tolist(), skip_special_tokens=True)

                # Print predicted and target answers
                print("Predicted Answer:", decoded_pred)
                print("Target Answer:", decoded_label)
                print("\n")

                # Append the decoded prediction and label for the current sample
                all_preds.append(decoded_pred)
                all_labels.append(decoded_label)

            except Exception as e:
                print(f"Error during evaluation for sample {i}: {e}")
                all_preds.append("")
                all_labels.append("")
                continue

    # Calculate exact match
    em = metric.compute(predictions=all_preds, references=all_labels)["exact_match"]

    print("\nEvaluation Results:")
    print(f"  Exact Match: {em:.4f}")

    return {"exact_match": em}


## WORKINg-2

In [ ]:
import torch
from tqdm import tqdm
import evaluate
from peft import PeftModel

def evaluate_model(model_path, eval_dataset, device="cuda" if torch.cuda.is_available() else "cpu", edges=None):
    """
    Evaluates a text-to-SQL model on a given dataset.

    Args:
        model_path (str): Path to the trained PEFT model.
        eval_dataset (Dataset): The dataset to evaluate on.
        device (str, optional): The device to use for computation ("cuda" or "cpu"). Defaults to "cuda" if available.
        edges (torch.Tensor, optional): Precomputed edge indices for the dataset (if applicable).

    Returns:
        dict: A dictionary containing the exact match score.
    """

    # Load model once outside the loop
    model = GraphModel(mistral_model, tokenizer)
    model = PeftModel.from_pretrained(model, model_path, is_trainable=False)
    model.to(device)
    model.float()
    model.eval()
    model.encoder.eval()

    metric = evaluate.load("exact_match")
    all_preds, all_labels = [], []

    with torch.no_grad():
        for i in tqdm(range(len(eval_dataset)), desc="Evaluating"):
            data = eval_dataset[i]

            Q = data['input_ids'].to(device).long()
            A = data['labels'].to(device).long()
            # Check for NaN or Inf values in Q and A
            if torch.isnan(Q).any() or torch.isinf(Q).any() or torch.isnan(A).any() or torch.isinf(A).any():
                print("WARNING: NaN or Inf values detected in input_ids or labels!")

            print('\n\n')
            print(f"Sample ID: {i}")
            print(f"  - Text: {tokenizer.decode(Q, skip_special_tokens=True)}")
            print(f"  - Target Text: {tokenizer.decode(A, skip_special_tokens=True)}")
            print('\n')

            # Prepare data for the model
            input_data = {
                "input_ids": Q.unsqueeze(0),
                "attention_mask": data['attention_mask'].unsqueeze(0).to(device).float()
            }

            if edges is None:
                edges = torch.tensor([[]]).to(device)

            try:
                # Model inference
                model_output = model(**input_data, edges=edges.float())
                logits = model_output.logits


                # Handle potential errors in logits
                if torch.isnan(logits).any() or torch.isinf(logits).any():
                    print("WARNING: NaN or Inf values detected in logits!")
                    logits[torch.isnan(logits)] = 0 # Replace NaN with zero
                    logits[torch.isinf(logits)] = 0 # Replace Inf with zero


                # Ensure logits has the right dimensions before argmax
                if logits.dim() > 2:
                    logits = logits.squeeze(1)
                elif logits.dim() == 1:
                    logits = logits.unsqueeze(0)

                # Extract predicted tokens directly on GPU
                predicted_ids = logits.argmax(dim=-1)

                # Print shapes and values for debugging
                print("\n\n")
                print("Logits length:", logits.shape[1])
                print("Logits shape:", logits.shape)
                print("Logits dtype:", logits.dtype)
                print("\n")
                print("Predicted IDs:", logits.argmax(dim=-1))
                print("Labels:", data['labels'])
                print("\n")

                # Decode predictions and labels (move to CPU for decoding)
                decoded_pred = tokenizer.decode(predicted_ids.cpu(), skip_special_tokens=True)
                decoded_label = tokenizer.decode(A.cpu(), skip_special_tokens=True)

                print("Predicted Answer:", decoded_pred)
                print("Target Answer:", decoded_label)
                print("\n")

                all_preds.append(decoded_pred)
                all_labels.append(decoded_label)

            except RuntimeError as e:
                if 'out of memory' in str(e):  # Check if it's an OOM error
                    torch.cuda.empty_cache()
                    print(f"Out of memory error for sample {i}. Skipping this sample.")
                else:
                    print(f"Error during evaluation for sample {i}: {e}")
                all_preds.append("")  # Add an empty string for the skipped sample
                all_labels.append("")

    # Calculate exact match
    em = metric.compute(predictions=all_preds, references=all_labels)["exact_match"]

    print("\nEvaluation Results:")
    print(f"  Exact Match: {em:.4f}")

    return {"exact_match": em}


In [ ]:
path="/content/gdrive/MyDrive/model/GNN-T2SQL/checkpoint-250"
!ls -ltha $path

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path="/content/gdrive/MyDrive/model/GNN-T2SQL/checkpoint-250"
results=evaluate_model(path, eval_dataset, device=device)

## WORK-4

In [17]:
import torch
from tqdm import tqdm
import evaluate
from peft import PeftModel

import transformers
#from transformers import MistralTokenizer, MistralForCausalLM
import datasets

#def evaluate_model(model_path, eval_dataset, device="cuda" if torch.cuda.is_available() else "cpu", edges=None, fake_input=None):

def evaluate_model(model_path, eval_dataset, device="cuda" if torch.cuda.is_available() else "cpu", edges=None, fake_input=None):
    """
    Evaluates a text-to-SQL model on a given dataset, with optional fake input for testing.

    Args:
        model_path (str): Path to the trained PEFT model.
        eval_dataset (Dataset): The dataset to evaluate on.
        device (str, optional): The device to use for computation ("cuda" or "cpu"). Defaults to "cuda" if available.
        edges (torch.Tensor, optional): Precomputed edge indices for the dataset (if applicable).
        fake_input (str, optional): Fake input text to use instead of the actual dataset input.

    Returns:
        dict: A dictionary containing evaluation metrics, including exact match (EM), BLEU score, execution accuracy (if applicable), and others.
    """

    # Load model once outside the loop
    model = GraphModel(mistral_model, tokenizer)
    model = PeftModel.from_pretrained(model, model_path, is_trainable=False)
    model.to(device)
    model.float()
    model.eval()
    model.encoder.eval()

    metric_em = evaluate.load("exact_match")
    metric_bleu = evaluate.load("bleu")
    all_preds, all_labels = [], []



    with torch.no_grad():
        # Only evaluate on one sample if using fake input
        for i in tqdm(range(1 if fake_input else len(eval_dataset)), desc="Evaluating"):
            if fake_input:
                Q = tokenizer(fake_input, return_tensors="pt")['input_ids'].to(device).long()
                A = torch.tensor([], device=device).long()  # Empty tensor for labels
            else:
                data = eval_dataset[i]
                Q = data['input_ids'].to(device).long()
                A = data['labels'].to(device).long()

            # Check for NaN or Inf values in Q and A
            if torch.isnan(Q).any() or torch.isinf(Q).any() or (not fake_input and (torch.isnan(A).any() or torch.isinf(A).any())):
                print("WARNING: NaN or Inf values detected in input_ids or labels!")
                continue  # Skip this sample if there are invalid values

            print('\n\n')
            print(f"Sample ID: {i}")
             # Convert Q to a NumPy array before decoding
            print(f"  - Text: {tokenizer.decode(Q.cpu().numpy()[0], skip_special_tokens=True)}")
            if not fake_input:
                print(f"  - Target Text: {tokenizer.decode(A, skip_special_tokens=True)}")
            print('\n')

             # Prepare data for the model
            input_data = {
                "input_ids": Q.unsqueeze(0),
                "attention_mask": data['attention_mask'].unsqueeze(0).to(device).float() if not fake_input else torch.ones_like(Q).unsqueeze(0)
            }

            if edges is None:
                edges = torch.tensor([[]]).to(device)

            try:
                # Model inference
                # Ensure edges is in the correct format (if it's being used by your model)
                if edges is not None:
                    # Example: If your model expects edges as a 2D tensor of shape (batch_size, num_edges)
                    edges = edges.unsqueeze(0)  # Add a batch dimension if necessary
                else:
                    edges = torch.empty((1, 0), dtype=torch.long, device=device) # Create an empty tensor if no edges are provided

                model_output = model(**input_data, edges=edges.float())
                logits = model_output.logits



                # Handle potential errors in logits
                if torch.isnan(logits).any() or torch.isinf(logits).any():
                    print("WARNING: NaN or Inf values detected in logits!")
                    logits[torch.isnan(logits)] = 0 # Replace NaN with zero
                    logits[torch.isinf(logits)] = 0 # Replace Inf with zero


                # Ensure logits has the right dimensions before argmax
                if logits.dim() > 2:
                    logits = logits.squeeze(1)
                elif logits.dim() == 1:
                    logits = logits.unsqueeze(0)

                # Extract predicted tokens directly on GPU
                predicted_ids = logits.argmax(dim=-1)

                # Convert to list only if it's not already a list
                if not isinstance(predicted_ids, list):
                    predicted_ids = predicted_ids.cpu().tolist()

                # Print the type and shape of predicted_ids for debugging
                print("Type of predicted_ids:", type(predicted_ids))
                print("Shape of predicted_ids:", predicted_ids.shape)


                # Print the shape of hidden_states for debugging
                #print("Shape of hidden_states:", hidden_states.shape)  # Add this line

                #bsz, q_len, _ = hidden_states.size()




                # Always convert predicted_ids to a list, regardless of its dimension
                #predicted_ids = predicted_ids.cpu().tolist()



                # Print shapes and values for debugging
                print("\n\n")
                print("Logits length:", logits.shape[1])
                print("Logits shape:", logits.shape)
                print("Logits dtype:", logits.dtype)
                print("\n")
                print("Predicted IDs:", predicted_ids)
                if not fake_input:
                    print("Labels:", data['labels'])
                print("\n")


                # Decode predictions and labels
                decoded_pred = tokenizer.decode(predicted_ids, skip_special_tokens=True) # Decode the predicted IDs



                if not fake_input:
                    decoded_label = tokenizer.decode(A.cpu(), skip_special_tokens=True)
                    all_labels.append(decoded_label)

                print("Predicted Answer:", decoded_pred)
                if not fake_input:
                    print("Target Answer:", decoded_label)
                print("\n")

                all_preds.append(decoded_pred)  # Append even if fake input

            except RuntimeError as e:
                if 'out of memory' in str(e):  # Check if it's an OOM error
                    torch.cuda.empty_cache()
                    print(f"Out of memory error for sample {i}. Skipping this sample.")
                else:
                    print(f"Error during evaluation for sample {i}: {e}")
                all_preds.append("")  # Add an empty string for the skipped sample
                if not fake_input:
                    all_labels.append("")

    # Calculate exact match
    results = {}
    if not fake_input:
        em = metric_em.compute(predictions=all_preds, references=all_labels)["exact_match"]
        results["exact_match"] = em
        # Calculate BLEU score
        bleu = metric_bleu.compute(predictions=all_preds, references=all_labels)["bleu"]
        results["bleu"] = bleu


        # Calculate execution accuracy (if applicable)
        # ... (You'll need to implement this based on your specific setup)

        print("\nEvaluation Results:")
        print(f"  Exact Match: {em:.4f}")
        print(f"  BLEU Score: {bleu:.4f}")
        # Print execution accuracy if calculated

    return results


In [18]:
# 1. Load and Prepare Data
dataset = load_dataset("b-mc2/sql-create-context")["train"].shuffle(seed=42)

# Manually define splits
train_size = int(0.7 * len(dataset))
eval_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - eval_size

train_dataset = dataset.select(range(train_size))
eval_dataset = dataset.select(range(train_size, train_size + eval_size))
test_dataset = dataset.select(range(train_size + eval_size, len(dataset)))

In [15]:
!pip install transformers peft datasets evaluate accelerate bitsandbytes -q

In [21]:
# Specify the path to your saved model
#model_path ="/content/gdrive/MyDrive/model/GNN-T2SQL/checkpoint-250"


model_path = "/content/gdrive/MyDrive/model/GNNT2SQL/checkpoint-1950"

#model_path ="/content/gdrive/MyDrive/model/GNNPOC-T2SQL/checkpoint-2500"

# Use the fake input
fake_input = "What is the capital of the USA?"

# Evaluate the model with the fake input
results = evaluate_model(model_path, test_dataset, fake_input=fake_input)

# Print the results
print(results)  # This will likely be an empty dictionary since there's no exact match to compare to


TypeError: evaluate_model() got an unexpected keyword argument 'fake_input'

## NOT WORK

In [ ]:
from torch.utils.data import DataLoader

path="/content/GNN-T2SQL/checkpoint-500"

# 1. Load the Entire Model (With Adapter)
model = GraphModel(mistral_model, tokenizer)
model = PeftModel.from_pretrained(model, path, is_trainable=False)
model.to(device)

# 3. Create DataLoader with Collator
data_collator = GraphDataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)


eval_dataloader = DataLoader(eval_dataset, batch_size=1, collate_fn=data_collator)


In [ ]:
data=eval_dataloader
print(data.dataset[0]['input_ids'])
data.dataset[0]['labels']

In [ ]:
#eval_dataloader
edges=None


# 2. Initialize Metrics
metric = evaluate.load("exact_match")
all_preds, all_labels = [], []

with torch.no_grad():
      #for i in tqdm(range(len(eval_dataset)), desc="Evaluating"):
      for batch in tqdm(eval_dataloader, desc="Evaluating"):  # Iterate over the DataLoader
          # Extract the data from the batch (assuming batch size 1)
          data = batch
          #data = eval_dataset[i]
          #print(data)

          Q = data['input_ids'].squeeze()  # Convert tensor to list
          A = data['labels'].squeeze()  # Convert tensor to list

          #Q = data['input_ids']
          #A = data['labels']

          print('\n\n')
          #print(f"Sample ID: {i}")
          print(f"  - Text: {tokenizer.decode(Q.tolist(), skip_special_tokens=True)}") # Decode after converting to list
          print(f"  - Target Text: {tokenizer.decode(A.tolist(), skip_special_tokens=True)}") # Decode after converting to list
          print('\n')


          # Prepare data for the model, move tensors to the correct device and unsqueeze
          data = {k: v.unsqueeze(0).to(device).float() for k, v in data.items() if isinstance(v, torch.Tensor)} # Apply unsqueeze only to tensors

          # Convert 'input_ids' to LongTensor
          data['input_ids'] = data['input_ids'].long()

          # Provide a default 'edges' tensor if None
          if edges is None:
              edges = torch.tensor([[]]).to(device)
          #edges=edges.float()

           # Prepare data for the model
          input_data = {
                "input_ids": torch.tensor(Q).unsqueeze(0).to(device).long(),  # Convert list to tensor and unsqueeze
                "attention_mask": torch.tensor(data['attention_mask']).unsqueeze(0).to(device).float()  # Convert list to tensor and unsqueeze
            }

          if edges is None:
              edges = torch.tensor([[]]).to(device)


          # Model inference
          #model_output = model(**input_data, edges=edges.float())
          #logits = model_output.logits



          model_output = model(input_ids=data['input_ids'], attention_mask=data['attention_mask'], edges=edges.float())
          logits = model_output.logits

          if logits.dim() > 2:
              logits = logits.squeeze(1)




          #all_preds.extend(logits.argmax(dim=-1).cpu().tolist())
          #all_labels.extend(A.cpu().tolist())

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import evaluate
from peft import PeftModel

def evaluate_model(model_path, eval_dataset, device=None, edges=None):

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1. Load the Entire Model (With Adapter)
    model = GraphModel(mistral_model, tokenizer)
    model = PeftModel.from_pretrained(model, model_path, is_trainable=False)
    model.to(device)

    # Explicitly set model to float32 to ensure consistency
    model.float()

    # 2. Initialize Metrics
    metric = evaluate.load("exact_match")
    all_preds, all_labels = [], []

    # 3. Create DataLoader with Collator
    data_collator = GraphDataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=-100,
        pad_to_multiple_of=8
    )
    eval_dataloader = DataLoader(eval_dataset, batch_size=1, collate_fn=data_collator)

    # 4. Evaluation Loop
    model.eval()
    model.encoder.eval()
    with torch.no_grad():
        #for i in tqdm(range(len(eval_dataset)), desc="Evaluating"):
        for batch in tqdm(eval_dataloader, desc="Evaluating"):  # Iterate over the DataLoader
            # Extract the data from the batch (assuming batch size 1)
            #data = batch[0]
            #data = eval_dataset[i]


            data = batch


            Q = data['input_ids']
            A = data['labels']

            print('\n\n')
            #print(f"Sample ID: {i}")
            #print(f"  - Text: {tokenizer.decode(Q, skip_special_tokens=True)}")
            #print(f"  - Target Text: {tokenizer.decode(A, skip_special_tokens=True)}")
            #print('\n')


           # Prepare data for the model, move tensors to the correct device and unsqueeze
            data = {k: v.unsqueeze(0).to(device).float() for k, v in data.items()}

            # Convert 'input_ids' to LongTensor
            data['input_ids'] = data['input_ids'].long()

            # Unsqueeze the 'edges' tensor to add a batch dimension
            if edges is None:
                edges = torch.tensor([[]]).unsqueeze(0).to(device)  # Add unsqueeze here


            # Model inference
            model_output = model(**data, edges=edges.float())  # Pass the entire data dictionary
            logits = model_output.logits

            if logits.dim() > 2:
                logits = logits.squeeze(1)

            all_preds.extend(logits.argmax(dim=-1).cpu().tolist())
            all_labels.extend(A.cpu().tolist())


    # 5. Compute and Print Metrics
    decoded_preds = tokenizer.batch_decode(all_preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(all_labels, skip_special_tokens=True)

    # Print for debugging
    print(f"Decoded predictions: {decoded_preds}")
    print(f"Decoded labels: {decoded_labels}")

    em = metric.compute(predictions=decoded_preds, references=decoded_labels)["exact_match"]

    print("\nEvaluation Results:")
    print(f"  Exact Match: {em:.4f}")

In [ ]:
eval_dataset.dataset[0]

In [ ]:
eval_dataset[0]

In [ ]:
Q=eval_dataset[0]['input_ids']
tokenizer.decode(Q, skip_special_tokens=True)

In [ ]:
A=eval_dataset[0]['labels']
tokenizer.decode(A, skip_special_tokens=True)

In [ ]:

# Data Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8  # Ensure tensors divisible by 8 for optimized performance
)

# 10. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)


Let's analyze the information you've provided about your `GraphModel`:

**Structure Analysis:**

* **Encoder (Mistral-7B):** The backbone of your model is the Mistral-7B language model. This model is responsible for understanding the input text and generating meaningful representations (embeddings) for each token. The encoder has a substantial number of parameters (7.5 billion), indicating its large size and capacity to learn complex language patterns.

* **GAT Layer:**  You've incorporated a Graph Attention Network (GAT) layer to process the graph structure of your input data. This layer likely learns to weigh the importance of different nodes and edges based on their relationships, which can help capture additional information beyond just the raw text.

* **Pooling:** The `pool` operation is used to aggregate the representations from individual nodes into a single representation for the entire graph. The choice of pooling (e.g., mean pooling) depends on your specific task and how you want to combine the node-level information.

* **LM Head:** The language model head (`lm_head`) takes the final graph representation and predicts the next token in the sequence. It has a large number of parameters due to the vocabulary size (32,768).

**PEFT (Parameter-Efficient Fine-Tuning):**

- **Lora (Low-Rank Adaptation):** You are using LoRA, a technique that adds small adapter modules to the model to fine-tune it more efficiently. This allows you to train a smaller number of parameters while still achieving good performance.

- **Trainable Parameters:**  Only a small fraction (0.0078%) of the total parameters are trainable, thanks to LoRA. This drastically reduces the computational and memory requirements during training.

**Potential Challenges and Considerations:**

- **Graph Structure Quality:** The effectiveness of the GAT layer heavily relies on the quality of the graph structure you create from the input text. Dependency parsing is a good starting point, but you might want to explore other ways to construct the graph to better capture the relationships between words and phrases.

- **Overfitting with LoRA:** Even with LoRA, overfitting can be a concern, especially with a small dataset. Monitor your validation loss and consider techniques like early stopping or adding more training data (if possible).

- **Memory Usage:**  Graph models can be memory intensive, especially with large input sequences or large batch sizes. You might need to experiment with techniques like gradient checkpointing or gradient accumulation to manage memory usage.

- **Training Stability:** Training large language models can sometimes be unstable. Experiment with different learning rates, optimizers, and warm-up strategies to find the best settings for your model.

- **Evaluation Metrics:** Choose appropriate evaluation metrics that reflect the quality of the generated SQL queries. Consider using both automated metrics (e.g., BLEU score, accuracy) and human evaluation to assess the model's performance.


Let me know if you have any specific questions or concerns!


* Original Code - TRAINING ONLY


TO BE TEST IT WORK FOR trainer = SFTTrainer( ; from trl import SFTTrainer;

max_seq_length = 2048 --- max sequence length for model and packing of the dataset

 https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Mistral_7B_Instruct_v0_1_for_text_to_SQL_EVALDATA.ipynb



In [ ]:
torch.cuda.empty_cache()

In [ ]:
# 8. Evaluation Metric (Semantic Similarity)
metric = evaluate.load("exact_match")
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = tokenizer.batch_decode(logits, skip_special_tokens=True)
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute embeddings for predictions and references
    prediction_embeddings = sentence_transformer_model.encode(predictions, convert_to_tensor=True)
    reference_embeddings = sentence_transformer_model.encode(references, convert_to_tensor=True)

    # Calculate cosine similarities
    cosine_similarities = util.cos_sim(prediction_embeddings, reference_embeddings)
    similarities = torch.diag(cosine_similarities).cpu().numpy()  # Extract similarities for corresponding pairs

    # Return average similarity as the metric
    em = metric.compute(predictions=predictions, references=references)["exact_match"]
    return {"semantic_similarity": np.mean(similarities), "exact_match": em}

# 9. Training Arguments and Trainer
training_args = TrainingArguments(
    "graph-T2SQL",
    logging_dir="graph-T2SQL",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    push_to_hub=False,
    dataloader_pin_memory=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_semantic_similarity",
    report_to="tensorboard",
)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Data Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8  # Ensure tensors divisible by 8 for optimized performance
)

print('\n\n')

# 10. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))


# 11. Train the model
print('\n\n')
trainer.train()
print('\n\n')

# 12. Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')
print(f'Test Exact Match: {test_results["eval_exact_match"]:.4f}')

## Enhance - 1: Data Augmentation

In [ ]:
!pip install nlpaug -q

In [ ]:
from nlpaug.augmenter.word import SynonymAug, RandomWordAug
from nlpaug.augmenter.sentence import RandomInsertionAug

def augment_data(dataset):
    augmented_data = []

    synonym_aug = SynonymAug(aug_src='wordnet')
    insert_aug = RandomInsertionAug(aug_p=0.1)  # Insert with 10% probability
    delete_aug = RandomWordAug(action="delete", aug_p=0.05)  # Delete with 5% probability

    for item in dataset:
        question = item['question']
        answer = item['answer']

        # Create augmented examples
        augmented_questions = [
            synonym_aug.augment(question),
            insert_aug.augment(question),
            delete_aug.augment(question)
        ]

        # Add original and augmented examples to the dataset
        for aug_question in augmented_questions:
            augmented_data.append({'question': aug_question, 'answer': answer})

    return augmented_data

# Augment the training dataset
train_dataset_augmented = augment_data(train_dataset)

## Enhance - 2: Alternative Architecture: Graph Transformer Network (GTN)

In [ ]:
!pip install torch_geometric -q

In [ ]:
from torch_geometric.nn import GTConv

class GTNModel(torch.nn.Module):
    def __init__(self, encoder):
        super(GTNModel, self).__init__()
        self.encoder = encoder
        self.gtn = GTConv(4096, 4096, num_layers=2)  # 2 GTN layers
        self.pool = lambda x, batch: torch.mean(x, dim=0, keepdim=True)
        self.lm_head = torch.nn.Linear(4096, tokenizer.vocab_size)

    def forward(self, **inputs):
        x = inputs['input_ids'].to(mistral_model.device)
        edge_index = inputs['edge_index'].to(mistral_model.device)
        embeddings = self.encoder(x).last_hidden_state.cpu()
        gtn_out = self.gtn(embeddings, edge_index.cpu())
        pooled = self.pool(gtn_out, inputs.get('batch', torch.zeros(gtn_out.size(0)).long()))
        out = self.lm_head(pooled.to(mistral_model.device))
        return {"logits": out}

# Replace the model with the GTNModel
model = GTNModel(mistral_model)

## Enhance - 3: Integration Enhance - 1 and Enhance - 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import spacy
import numpy as np
from torch_geometric.nn import GTConv  # For GTN
from trl import setup_chat_format
from nlpaug.augmenter.word import SynonymAug, RandomWordAug
from nlpaug.augmenter.sentence import RandomInsertionAug

# ... (rest of the imports and model/tokenizer loading from the original code)

# Data Augmentation Function
def augment_data(dataset):
    augmented_data = []

    synonym_aug = SynonymAug(aug_src='wordnet')
    insert_aug = RandomInsertionAug(aug_p=0.1)  # Insert with 10% probability
    delete_aug = RandomWordAug(action="delete", aug_p=0.05)  # Delete with 5% probability

    for item in dataset:
        question = item['question']
        answer = item['answer']

        # Create augmented examples
        augmented_questions = [
            synonym_aug.augment(question),
            insert_aug.augment(question),
            delete_aug.augment(question)
        ]

        # Add original and augmented examples to the dataset
        for aug_question in augmented_questions:
            augmented_data.append({'question': aug_question, 'answer': answer})

    return augmented_data

# TextToSQLDataset (unchanged from the original code)
class TextToSQLDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        #text = item['question'] + " " + item['context']
        text = item['question']
        target_text = item['answer']

        tokenized_input = self.tokenizer(text, truncation=True, padding=True, return_tensors="pt")
        tokenized_target = self.tokenizer(target_text, truncation=True, padding=True, return_tensors="pt")

        # Dependency Parsing for Edge Index
        doc = nlp(text)
        edges = []
        for token in doc:
            if token.i < len(doc) - 1:
                edges.append([token.i, token.head.i])
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        return {
            'input_ids': tokenized_input['input_ids'].flatten(),
            'attention_mask': tokenized_input['attention_mask'].flatten(),
            'labels': tokenized_target['input_ids'].flatten(),
            'edge_index': edge_index,
        }

# GTN Model WITH GTConv
from torch_geometric.nn import GTConv

class GTNModel(torch.nn.Module):
    def __init__(self, encoder):
        super(GTNModel, self).__init__()
        self.encoder = encoder
        self.gtn = GTConv(4096, 4096, num_layers=2)  # 2 GTN layers
        self.pool = lambda x, batch: torch.mean(x, dim=0, keepdim=True)
        self.lm_head = torch.nn.Linear(4096, tokenizer.vocab_size)

    def forward(self, **inputs):
        x = inputs['input_ids'].to(mistral_model.device)
        edge_index = inputs['edge_index'].to(mistral_model.device)
        embeddings = self.encoder(x).last_hidden_state.cpu()

        gtn_out = self.gtn(embeddings, edge_index.cpu())
        pooled = self.pool(gtn_out, inputs.get('batch', torch.zeros(gtn_out.size(0)).long()))
        out = self.lm_head(pooled.to(mistral_model.device))
        return {"logits": out}

# Replace the model with the GTNModel
model = GTNModel(mistral_model)
model = get_peft_model(model, peft_config)
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint

# 7. Evaluation Metric (Semantic Similarity)
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = tokenizer.batch_decode(logits, skip_special_tokens=True)
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Compute embeddings for predictions and references
    prediction_embeddings = sentence_transformer_model.encode(predictions, convert_to_tensor=True)
    reference_embeddings = sentence_transformer_model.encode(references, convert_to_tensor=True)

    # Calculate cosine similarities
    cosine_similarities = util.cos_sim(prediction_embeddings, reference_embeddings)
    similarities = torch.diag(cosine_similarities).cpu().numpy()  # Extract similarities for corresponding pairs

    # Return average similarity as the metric
    return {"semantic_similarity": np.mean(similarities)}


# Augment the training dataset
train_dataset_augmented = augment_data(train_dataset)

# Create datasets (using augmented training data)
train_dataset = TextToSQLDataset(train_dataset_augmented, tokenizer)
val_dataset = TextToSQLDataset(val_dataset, tokenizer)
test_dataset = TextToSQLDataset(test_dataset, tokenizer)

# Use the GTNModel
model = GTNModel(mistral_model)

model = get_peft_model(model, peft_config)
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint

torch.cuda.empty_cache()

# 10. Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator=lambda x: x,
    compute_metrics = compute_metrics,
    optimizers=(optimizer, None)
)

from transformers import EarlyStoppingCallback
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))

# 11. Train the model
trainer.train()

# 12. Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')

## Enhance - 4: Regularization Techniques

In [ ]:
# ... (other imports)
from torch_geometric.nn import GTConv

# ... (other classes and functions)

# GTN Model with Dropout
class GTNModel(torch.nn.Module):
    def __init__(self, encoder, dropout_prob=0.1):  # Add dropout probability
        super(GTNModel, self).__init__()
        self.encoder = encoder
        self.gtn = GTConv(4096, 4096, num_layers=2)
        self.dropout = nn.Dropout(dropout_prob)  # Dropout layer
        self.pool = lambda x, batch: torch.mean(x, dim=0, keepdim=True)
        self.lm_head = torch.nn.Linear(4096, tokenizer.vocab_size)

    def forward(self, **inputs):
        # ... (rest of the forward method)
        x = inputs['input_ids'].to(mistral_model.device)
        edge_index = inputs['edge_index'].to(mistral_model.device)
        embeddings = self.encoder(x).last_hidden_state.cpu()


        gtn_out = self.gtn(embeddings, edge_index.cpu())
        gtn_out = self.dropout(gtn_out)  # Apply dropout after GTN

        pooled = self.pool(gtn_out, inputs.get('batch', torch.zeros(gtn_out.size(0)).long()))

        out = self.lm_head(pooled.to(mistral_model.device))
        return {"logits": out}

# ... (rest of the code)

# Create the model with dropout
model = GTNModel(mistral_model, dropout_prob=0.1)  # Set dropout probability
model = get_peft_model(model, peft_config)


model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint

torch.cuda.empty_cache()

# 10. Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator=lambda x: x,
    compute_metrics = compute_metrics,
    optimizers=(optimizer, None)
)

from transformers import EarlyStoppingCallback
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))

# 11. Train the model
trainer.train()

# 12. Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')


## Enhance - 5:



* MultiTaskDataset: This class combines multiple datasets and assigns task IDs to each sample.

* MultiTaskModel: This class extends the model to include a separate head for the classification task.

* Task-Specific Forward Pass: The forward method now takes a task_id input and uses the appropriate head based on the task.

* Training Loop: The training loop needs to be modified to handle the multi-task dataset and compute losses for both tasks.

In [ ]:
class TextToSQLDataset(Dataset):
    def __init__(self, dataset, tokenizer, task_id=0):  # Add task_id argument
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.task_id = task_id  # Store task ID

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        question = item['question'] if self.task_id == 0 else item['text']  # Use appropriate field
        answer = item['answer'] if self.task_id == 0 else item['label']  # Use appropriate field
        # ... (rest of the __getitem__ method for tokenization and graph construction)


        item = {'input_ids': input_ids, 'attention_mask': attention_mask,
                'edge_index': edge_index, 'task_id': self.task_id}  # Add task_id
        return item


In [ ]:
# Create the multi-task model with dropout
model = MultiTaskModel(mistral_model, dropout_prob=0.1, num_classes=num_classification_classes)
model = get_peft_model(model, peft_config)

In [ ]:
# ... (other parts of the code)

train_dataset_sql = TextToSQLDataset(train_dataset_augmented, tokenizer, task_id=0)
train_dataset_classification = TextToSQLDataset(dataset_classification, tokenizer, task_id=1)

# Combine datasets (consider balancing them if needed)
train_dataset = torch.utils.data.ConcatDataset([train_dataset_sql, train_dataset_classification])

# ... (rest of the training and evaluation code, modified for multi-task)


In the provided code, the new train_dataset brings several enhancements compared to the original version:

Data Augmentation: The most significant change is the incorporation of data augmentation. The original train_dataset is passed through the augment_data function, which creates additional training examples by applying techniques like synonym replacement, random word insertion, and random word deletion. This augmented dataset is then used to create the new train_dataset.

Benefit: Data augmentation helps the model generalize better to different phrasing and vocabulary variations in natural language input, leading to improved performance on unseen examples.
Multi-Task Learning: The new train_dataset is designed to support multi-task learning. It combines examples from both the original text-to-SQL task and an additional classification task. Each example in the dataset is associated with a task_id to indicate which task it belongs to.

Benefit: Multi-task learning allows the model to learn shared representations between related tasks, potentially improving performance and generalization on both tasks.
Dynamic Padding: While not explicitly mentioned, the code likely uses dynamic padding when creating the train_dataset. This means that each example is padded to the length of the longest sequence in its batch, rather than using a fixed maximum length.

Benefit: Dynamic padding reduces unnecessary padding tokens, which can speed up training and potentially improve model performance.

Sources and related content


## Enhance - 6: Custom Loss Function

In [ ]:
from transformers import EarlyStoppingCallback
from torch.nn import CrossEntropyLoss, MSELoss


# Custom Loss Function (integrated)
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    task_ids = inputs.pop("task_id")

    outputs = model(**inputs)

    sql_logits = outputs.logits[task_ids == 0]  # SQL task logits
    sql_labels = labels[task_ids == 0]          # SQL task labels
    sql_loss = CrossEntropyLoss()(sql_logits, sql_labels)

    classification_logits = outputs.logits[task_ids == 1]  # Classification task logits
    classification_labels = labels[task_ids == 1]          # Classification task labels
    classification_loss = MSELoss()(classification_logits, classification_labels)

    # Combine Losses (adjust weights as needed)
    loss = 0.5 * sql_loss + 0.5 * classification_loss

    return (loss, outputs) if return_outputs else loss

# 10. Create Trainer instance (with the custom loss)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda x: x,
    compute_metrics=compute_metrics,
    compute_loss=compute_loss,  # Pass the custom loss function here
    optimizers=(optimizer, None)
)

# 11. Train the model
trainer.train()

# 12. Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')